In [ ]:
pip install geopy


In [1551]:
import pandas as pd 
import numpy as np 
import math 
import os
import datetime
from datetime import date, datetime, timedelta
import re
import ast
import geopy
from geopy import distance

from geopy.geocoders import Nominatim


## 1.Checking dimensions tables.

In [1202]:
dimension_table = pd.read_excel('Inputs/User Details.xlsx', sheet_name = None,  parse_dates=True )

In [1203]:
if 'pandas' in str(type(dimension_table)): 
    print(dimension_table)
elif 'dict' in str(type(dimension_table)):
    for dimension in dimension_table:
        print('---------------------------------------------------------------------------------------')
        print(dimension)
        print('---------------------------------------------------------------------------------------')
        print(dimension_table[dimension])
        print('---------------------------------------------------------------------------------------')

---------------------------------------------------------------------------------------
Accounts
---------------------------------------------------------------------------------------
   AccountID AccountName
0         55      AMDS 1
1         25      AMDS 2
2         71      AMDS 3
3         58      AMDS 4
4         48      AMDS 5
5         81      AMDS 6
6         25      AMDS 7
7         61      AMDS 8
8         26      AMDS 9
---------------------------------------------------------------------------------------
---------------------------------------------------------------------------------------
Assets
---------------------------------------------------------------------------------------
   AssetID License Plate     Make Model  AccountID  Datasource  Tank Capacity
0      362    GA-567-789   Renaut  R-1A         55      334455            100
1      289    GA-123-456     Ford  F-2B         55      334456             90
2      338    EU-234-567  Citroen  C-3C         55      3344

In [1204]:
accounts = dimension_table['Accounts']
assets = dimension_table ['Assets']



In [1205]:
print(accounts.info())
print('--------------------------------------------')
print(accounts.duplicated(subset='AccountID').sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   AccountID    9 non-null      int64 
 1   AccountName  9 non-null      object
dtypes: int64(1), object(1)
memory usage: 276.0+ bytes
None
--------------------------------------------
1


<b>So, there are duplicates</b> in primary key for Accounts table. Not good. Should be reported to developers. 
Can be resolved in many ways: 
- 1.Try to find out which AccountID should actually have each of accounts in alternative source of truth (if there is any), report to devs and to fix it for now as ad hoc. 
- 2. Drop duplicate (second one by default). 
- 3. To set it last existing id in current table +1 at least for now in case I need to calculate some stats in account table. 

Let's have a look on, probably, connected assets table though. 

Datatypes looks right. 


In [1206]:
assets.shape[0]

5

In [1207]:
print(assets.info())
print()
print(assets.nunique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   AssetID        5 non-null      int64 
 1   License Plate  5 non-null      object
 2   Make           5 non-null      object
 3   Model          5 non-null      object
 4   AccountID      5 non-null      int64 
 5   Datasource     5 non-null      int64 
 6   Tank Capacity  5 non-null      int64 
dtypes: int64(4), object(3)
memory usage: 412.0+ bytes
None

AssetID          5
License Plate    5
Make             3
Model            5
AccountID        3
Datasource       5
Tank Capacity    4
dtype: int64


In [1208]:
def deduplicator(df, primary_keys): 
    if df.duplicated(subset = primary_keys).sum() > 0: 
        df.drop_duplicates(subset = primary_keys, inplace = True, ignore_index= True)
        return None #todo: write amount of deduplicated rows in a list -part of dict, which will be then dict of metadata. 

In [1209]:
deduplicator(accounts, 'AccountID')

In [1210]:
deduplicator(assets, 'AssetID')

In [1211]:
def values_mapper(df, column_name, mapper):
    new_col = column_name + '_fixed'
    df[new_col] = df[column_name].map(mapper)
    df[new_col] = df.apply(lambda x: x[column_name] if pd.isna(x[new_col]) else x[new_col], axis =1 )

In [1212]:
assets_fix = {'Renaut': 'Renault'}
values_mapper(assets, 'Make', assets_fix)
print(assets)
              

   AssetID License Plate     Make Model  AccountID  Datasource  Tank Capacity  \
0      362    GA-567-789   Renaut  R-1A         55      334455            100   
1      289    GA-123-456     Ford  F-2B         55      334456             90   
2      338    EU-234-567  Citroen  C-3C         55      334457            110   
3      485    AB-345-678   Renaut  R-2A         58      334458            120   
4      409    CA-456-789     Ford  F-3B         48      334459            100   

  Make_fixed  
0    Renault  
1       Ford  
2    Citroen  
3    Renault  
4       Ford  


In [1213]:
assets.rename(columns = {'Make': "maker", 'Make_fixed': 'maker_fixed', 'AssetID': 'asset_id', 
                         'License Plate': 'license_plate', 'Model': 'model', 'AccountID':'account_id', 
                         'Datasource': 'data_source', 'Tank Capacity': 'tank_capacity'
                        }, inplace = True)

In [1214]:
accounts.rename(columns= {'AccountID': 'account_id', 'AccountName':'account_name'}, inplace = True)

In [1215]:
accounts

,account_id,account_name
0,55,AMDS 1
1,25,AMDS 2
2,71,AMDS 3
3,58,AMDS 4
4,48,AMDS 5
5,81,AMDS 6
6,61,AMDS 8
7,26,AMDS 9


In [1216]:
print(accounts.info())
print()
print('----------------')
print()
print(assets.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   account_id    8 non-null      int64 
 1   account_name  8 non-null      object
dtypes: int64(1), object(1)
memory usage: 260.0+ bytes
None

----------------

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   asset_id       5 non-null      int64 
 1   license_plate  5 non-null      object
 2   maker          5 non-null      object
 3   model          5 non-null      object
 4   account_id     5 non-null      int64 
 5   data_source    5 non-null      int64 
 6   tank_capacity  5 non-null      int64 
 7   maker_fixed    5 non-null      object
dtypes: int64(4), object(4)
memory usage: 452.0+ bytes
None


Talbes cleared. Let.s leave it


## 2. Facts tables cleaning

In [1397]:
def fact_files_to_read(input_directory):
    objects = os.scandir(inputs_dir)
    fileList = []
    for obj in objects:
        if obj.is_file() and re.fullmatch('\d*\.xlsx', obj.name) is not None:
            fileList.append(inputs_dir+obj.name)
    return fileList 

In [1398]:
inputs_dir= 'Inputs/'
files = fact_files_to_read(inputs_dir)
files

['Inputs/334455.xlsx',
 'Inputs/334456.xlsx',
 'Inputs/334457.xlsx',
 'Inputs/334458.xlsx',
 'Inputs/334459.xlsx']

In [1399]:
def facts_reader(files): 
    dfs= []
    for file in files: 
        df=pd.read_excel(file, parse_dates=[1])
        dfs.append(df)
    return dfs 
        

In [1400]:
fact_tables = facts_reader(files)

In [1401]:
for fact_table in fact_tables:
    print(fact_table.columns)
    print('-------')
    print()   
#Typos in column names (whitespaces), also different names for some reason. And Odometer Mts and without Mts, Pos (lat, lon)
#and without (Lat, Long) - different formats? 
#Timestamp with format only in first file. Need to check the data and it's datatypes then. 

Index(['ID', ' Timestamp (dd/mm/yyyy hh:mm:ss)', ' Ignition', ' Odometer',
       ' Pos (Lat, Long)', ' Fuel %age'],
      dtype='object')
-------

Index(['ID', ' Timestamp', ' Ignition', ' Odometer Mts', ' Pos', ' Fuel %age'], dtype='object')
-------

Index(['ID', ' Timestamp', ' Ignition', ' Odometer Mts', ' Pos', ' Fuel %age'], dtype='object')
-------

Index(['ID', ' Timestamp', ' Ignition', ' Odometer', ' Pos', ' Fuel %age'], dtype='object')
-------

Index(['ID', ' Timestamp', ' Ignition', ' Odometer', ' Pos', ' Fuel %age'], dtype='object')
-------



In [1402]:
#Visually check data and data types. 
for fact_table in fact_tables:
    print(fact_table.head())
    print('-------------------')
    print()
    print('-------------------')
# See different column names. Needs to be fixed. 
#also - unexpected nulls in ignition for 334455, 334458 , odometer nulls, fueel %age. 

       ID  Timestamp (dd/mm/yyyy hh:mm:ss)   Ignition   Odometer  \
0  334455              2024-01-01 15:00:00        1.0        NaN   
1  334455              2024-01-01 15:04:00        NaN        NaN   
2  334455              2024-01-01 15:07:00        NaN        NaN   
3  334455              2024-01-01 15:09:00        NaN        NaN   
4  334455              2024-01-01 15:12:00        NaN        NaN   

         Pos (Lat, Long)   Fuel %age  
0  (53.332025,-6.275705)       100.0  
1   (53.331461,-6.30369)         NaN  
2  (53.330896,-6.331675)         NaN  
3   (53.330332,-6.35966)         NaN  
4  (53.329767,-6.387645)         NaN  
-------------------

-------------------
       ID           Timestamp   Ignition   Odometer Mts  \
0  334456 2024-01-04 10:00:00          1           1.48   
1  334456 2024-01-04 10:02:08          1           2.96   
2  334456 2024-01-04 10:02:37          1           4.44   
3  334456 2024-01-04 10:03:51          1           5.92   
4  334456 2024-01-04 

In [1403]:
fact_tables[0].info() #some of pos are nulls, most of ignition, fueld, odometer values are nan. Need to fix or drop nulls in the future.
#assumption: pair of ID+timestamp = natural key. 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 6 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   ID                                204 non-null    int64         
 1    Timestamp (dd/mm/yyyy hh:mm:ss)  204 non-null    datetime64[ns]
 2    Ignition                         6 non-null      float64       
 3    Odometer                         18 non-null     float64       
 4    Pos (Lat, Long)                  202 non-null    object        
 5    Fuel %age                        6 non-null      float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(1)
memory usage: 9.7+ KB


In [1404]:
print(fact_tables[0][' Ignition'].dropna().values) #integers, but not so many. 
print('-------------')
print()
print('-------------')
print(fact_tables[0][' Odometer'].dropna().values)  # so,integers in reality, floats in table with  in the middle for some reason. 
print('-------------')
print()
print('-------------')
print(fact_tables[0][' Fuel %age'].dropna().values) #integers in reality, float in table, but not so many. 

[1. 1. 0. 1. 1. 0.]
-------------

-------------
[ 15.  19.  41.   0.  20.  34.  66.  96. 118. 148. 149. 176. 210.   2.
  38.  74. 114. 134.]
-------------

-------------
[100.  80.  60.  40. 100.  80.]


In [1405]:
fact_tables[0].info() #at least no nulls. 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 6 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   ID                                204 non-null    int64         
 1    Timestamp (dd/mm/yyyy hh:mm:ss)  204 non-null    datetime64[ns]
 2    Ignition                         6 non-null      float64       
 3    Odometer                         18 non-null     float64       
 4    Pos (Lat, Long)                  202 non-null    object        
 5    Fuel %age                        6 non-null      float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(1)
memory usage: 9.7+ KB


In [1406]:
print(fact_tables[1][' Ignition'].dropna().values) #ingegers
print('-------------')
print()
print('-------------')
print(fact_tables[1][' Odometer Mts'].dropna().values)  # floats 
print('-------------')
print()
print('-------------')
print(fact_tables[1][' Fuel %age'].dropna().values) #integers


#also observed all other files in the same way and...

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
-------------

-------------
[  1.48   2.96   4.44   5.92   7.4    8.88  10.36  11.84  13.32  14.8
  16.28  17.76  19.24  20.72  22.2   23.68  25.16  26.64  28.12  29.6
  31.08  32.56  34.04  35.52  37.    38.48  39.96  41.44  42.92  44.4
  45.88  47.36  48.84  50.32  51.8   53.28  54.76  56.24  57.72  59.2
  60.68  62.16  63.64  65.12  66.6   68.08  69.56  71.04  72.52  74.
  75.48  76.96  78.44  79.92  81.4   82.88  84.36  85.84  87.32  88.8
  90.28  91.76  93.24  94.72  96.2   97.

### 2.1 Settings common columns names. 

In [1407]:
#... and now I can at least rename columns for all files and to convert datatypes to the most suitable
#wide type: if the data in one file in column_x is int and in another file is float - it'll be float, as a type, preserving 
#accuracy of both floats and ints. 
def columns_common_names(list_df, cols_list):
    for df in list_df: 
        df.columns = cols_list
    return None
# To convert cols to common datatypes, but after I can get rid of nans. 
def columns_common_datatypes(list_df, cols_convert_map):
    for df in list_df: 
        for col in cols_convert_map:
            df[col]=df[col].apply(cols_convert_map[col])
    return None

In [1408]:
facts_cals = ['id', 'timestamp', 'ignition', 'odometer_m', 'pos', 'fuel_%']
convert_map = {
    'ignition': int, 
    'odometer_m': float, 
    'fuel_%': int    
}

columns_common_names(fact_tables, facts_cals)
#columns_common_datatypes(fact_tables, convert_map)

In [1409]:
for fact in fact_tables: 
    print(sum(fact.duplicated(subset=['id', 'timestamp']))) 
#quite unexpected. Means - id+timestamp isn't unique and 
#we actually can recieve >1 events in the same second. Can be resolved either by bigger granularity scale (milliseconds)
#or to add events_sequence. Let's check if id+timestamp+ignition are unique. 

2
0
2
2
2


In [1410]:
for fact in fact_tables: 
    print(sum(fact.duplicated(subset=['id', 'timestamp', 'ignition'])))
#yes, at least it's totally unique. But as there are null values for ignition in some tables, makes sense to fill in the values
#based either on: previous(initial) state. Or: initial state + movement fact (to check delta of distance between rows, if 
#distance covered >0, then ignition =1, if =0 - then...well, then it can be either ignition = 0 as well as ignition = 1 (you still
#can leave the ignited car or stay in traffic jam.)). 

#I think we can believe ignition sensors work, but work in different way, so in some cases they're producing results constantly,
#and sometimes - ony during state changes. 

0
0
0
0
0


In [1411]:
first_sample = fact_tables[0]
first_sample['is_ignition_value']=first_sample['ignition'].apply(lambda x: 0 if pd.isna(x) else 1)#could be fill na straightforward
#but decided to implement through table functions

In [1412]:
#first_sample['ignition_parts']= first_sample.sort_values('timestamp',  ascending=True).groupby(['id'])['ignition'].rank()



# first_sample['ignition_parts']=first_sample.sort_values('timestamp')['is_ignition_value'].cumsum()
# first_sample['ignition_filled'] = first_sample.sort_values('timestamp').groupby(['id','ignition_parts'])['ignition'].ffill()
# first_sample['ignition_filled'] = first_sample['ignition_filled'].apply(int)


# first_sample['prev_ignition'] = first_sample.sort_values('timestamp').groupby(['id'])\
# ['ignition_filled'].shift(1)
# first_sample['changed_ignition_status'] = first_sample['ignition_filled']!=first_sample['prev_ignition']
# first_sample['seq_number']=first_sample.sort_values(['timestamp', 'changed_ignition_status']).groupby(['id', 'timestamp']).cumcount()+1

# first_sample=first_sample[['id', 'timestamp', 'seq_number', 'ignition', 'ignition_filled', 'odometer_m', 'pos', 'fuel_%']]

### 2.2. Forward-filling ignition data

In [1413]:
def order_and_ignition_fill (df): 
    df['is_ignition_value']=df['ignition'].apply(lambda x: 0 if pd.isna(x) else 1)
    df['ignition_parts']= df.sort_values('timestamp')['is_ignition_value'].cumsum()
    df['ignition_filled'] = df.sort_values('timestamp').groupby(['id','ignition_parts'])['ignition'].ffill()
    df['ignition_filled'] = df['ignition_filled'].apply(int)
    
    #Sequence for the events, so engine_off should always be after engine on sent in the same time. 
    df['prev_ignition'] = df.sort_values('timestamp').groupby(['id'])['ignition_filled'].shift(1)
    df['changed_ignition_status'] = df['ignition_filled']!=df['prev_ignition']
    df['seq_number']=df.sort_values(['timestamp', 'changed_ignition_status']).groupby(['id', 'timestamp']).cumcount()+1
    return None





In [1414]:
def ignition_fill_all_tables(dflist):
    for df in dflist: 
        order_and_ignition_fill(df)
    return None

ignition_fill_all_tables(fact_tables)
for i, df in enumerate(fact_tables): 
    fact_tables[i]=fact_tables[i][['id', 'timestamp', 'seq_number', 'ignition', 'ignition_filled', 'odometer_m', 'pos', 'fuel_%']]


### 2.3. Forward filling the same fuel, odometer, geo for events occured the same second (last ignited moment+engine off)

In [1415]:
##Also it's possible to fill in coordinates and fuel, as well as odometer metric (should be nearly the same in the same moment of time) for those stop engine events, which happened the same second we have data for 
##start engine:
def fill_in_pos_fuel_od_ign_off(df):
    df['pos']=df.sort_values(['id','timestamp', 'seq_number']).groupby(['id', 'timestamp'])['pos'].ffill()
    df['fuel_%']=df.sort_values(['id','timestamp', 'seq_number']).groupby(['id', 'timestamp'])['fuel_%'].ffill()
    df['odometer_m']=df.sort_values(['id','timestamp', 'seq_number']).groupby(['id', 'timestamp'])['odometer_m'].ffill()
    return None
    
def fill_same_second_fuel_pos_gaps(dflist):
    for df in dflist:
        fill_in_pos_fuel_od_ign_off(df)
    return None

In [1416]:
fill_same_second_fuel_pos_gaps(fact_tables)

C:\Users\Yevhen\AppData\Local\Temp\ipykernel_17496\2045819370.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pos']=df.sort_values(['id','timestamp', 'seq_number']).groupby(['id', 'timestamp'])['pos'].ffill()
C:\Users\Yevhen\AppData\Local\Temp\ipykernel_17496\2045819370.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['fuel_%']=df.sort_values(['id','timestamp', 'seq_number']).groupby(['id', 'timestamp'])['fuel_%'].ffill()
C:\Users\Yevhen\AppData\Local\Temp\ipykernel_17496\2045819370.py:6: Sett

In [1417]:
for df in fact_tables:
    print(df.info())
    print()
    print('-------------------------------------')
    print()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id               204 non-null    int64         
 1   timestamp        204 non-null    datetime64[ns]
 2   seq_number       204 non-null    int64         
 3   ignition         6 non-null      float64       
 4   ignition_filled  204 non-null    int64         
 5   odometer_m       19 non-null     float64       
 6   pos              204 non-null    object        
 7   fuel_%           7 non-null      float64       
dtypes: datetime64[ns](1), float64(3), int64(3), object(1)
memory usage: 12.9+ KB
None

-------------------------------------

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id               252 non

### 2.4 Extracting geo data to recover then delta distance covered between two points. 

In [1418]:
#Okay, so ignition status is recovered. All the events have geo, but not all have odometer values. So, it makes sense to recover
#distance covered from geo. Will be alternative source of truth for o distance covered metric, as well, as a control metric for
#odometer. 

#And....let's put  latitude and longitude in separate columns too!


In [1419]:
#fact_tables[0]['prev_pos']=fact_tables[0].sort_values(['id', 'timestamp', 'seq_number']).groupby('id')['pos'].shift()

In [1420]:
def geo_extractor (string_tuple):
    """Allows to extract latitude and longitude from 
    stringified tuples of pos column. 
    """
    lat = np.nan
    lon = np.nan
    try: 
        lat = ast.literal_eval(string_tuple)[0]
        lon = ast.literal_eval(string_tuple)[1]
    except ValueError:
        print('Input is not a tuple')
    return lat, lon
    
def cleared_geo_tuple (lat, lon):
    if not (np.isnan(lat)) and not(np.isnan(lon)):
        return (lat,lon)
    else: 
        return np.nan
    

def prev_pos (df): 
    """Creates geo-cols in dataframe"""
    df['prev_pos'] = df.sort_values(['id', 'timestamp', 'seq_number']).groupby('id')['pos'].shift()
    df['current_lat'] = df['pos'].apply(lambda x: geo_extractor(x)[0])
    df['current_lon'] = df['pos'].apply(lambda x: geo_extractor(x)[1])
    df['prev_lat'] = df['prev_pos'].apply(lambda x: geo_extractor(x)[0])
    df['prev_lon'] = df['prev_pos'].apply(lambda x: geo_extractor(x)[1])
    df['current_loc']= df.apply(lambda x: cleared_geo_tuple(x['current_lat'], x['current_lon']), axis=1)
    df['prev_loc'] = df.apply(lambda x: cleared_geo_tuple(x['prev_lat'], x['prev_lon']), axis=1)
    #df['current_lon'] = df['pos'].apply(lambda x: ast.literal_eval(x)[1])
    #df['prev_lat'] = df['prev_pos'].apply(lambda x: ast.literal_eval(x)[0])
    #df['prev_lon'] = df['[prev_pos'].apply(lambda x: ast.literal_eval(x)[1])
    
    #df['col_3'] = df.apply(lambda x: get_sublist(x.col_1, x.col_2), axis=1)

    return None
    
def coordinates_parser(df_list): 
    """Iterates through the list of DFs to apply functions"""
    for df in df_list: 
        prev_pos(df)
    return None

In [1421]:
coordinates_parser(fact_tables)

C:\Users\Yevhen\AppData\Local\Temp\ipykernel_17496\1977845144.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['prev_pos'] = df.sort_values(['id', 'timestamp', 'seq_number']).groupby('id')['pos'].shift()
C:\Users\Yevhen\AppData\Local\Temp\ipykernel_17496\1977845144.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['current_lat'] = df['pos'].apply(lambda x: geo_extractor(x)[0])
C:\Users\Yevhen\AppData\Local\Temp\ipykernel_17496\1977845144.py:25: SettingWithCopyWarning: 
A value is trying to be s

Input is not a tuple
Input is not a tuple
Input is not a tuple
Input is not a tuple
Input is not a tuple
Input is not a tuple
Input is not a tuple
Input is not a tuple
Input is not a tuple
Input is not a tuple


### 2.5 Geo distances extraction for separate tables

In [1422]:
#Let's calculate distance by gps, as there is no gaps for gps location data. 

def gps_distance_for_all_tables (df_list):
    """Takes all dataframes and calculates gps distance for all of them"""
    def gps_distance_calc(curr_loc, prev_loc, index_of_df):
        try:
            #if not(np.isnan(curr_loc) or np.isnan(prev_loc):
            dst = distance.distance(curr_loc, prev_loc).m
        except ValueError: 
            print(f'Distance variable(s) is/are incorrect in {index_of_df} df')
            dst = np.nan
        return dst
    for i, df in enumerate(df_list): 
        df['gps_distance'] = df.sort_values(['id', 'timestamp']).apply(lambda x: gps_distance_calc(x['current_loc'], x['prev_loc'], i), axis= 1)
    return None 





#fact_tables[0]['distance'] = fact_tables[0].sort_values(['id','timestamp']).apply(lambda x: distance_calc(x['current_loc'], x['prev_loc']), axis= 1)

In [1423]:
# def gps_distance_calc(curr_loc, prev_loc):
#     dst = 0
#     try:
#         #if not(np.isnan(curr_loc) or np.isnan(prev_loc.all())):
#         dst = distance.distance(curr_loc, prev_loc).m
#     except ValueError: 
#         print('Distance variable(s) is/are incorrect')
#     return dst


# for i,df in enumerate(fact_tables): 
#     print(i)
#     df['gps_distance'] = df.sort_values(['id', 'timestamp']).apply(lambda x: gps_distance_calc(x['current_loc'], x['prev_loc']), axis= 1)
    
    
#warning can tell, that coordinates aren't correct at all. 

In [1424]:
gps_distance_for_all_tables(fact_tables)
#warning occured because of df #2, but it's okay, as values of distance will be nulls 
#there in this case, will be handled later. 

Distance variable(s) is/are incorrect in 0 df
Distance variable(s) is/are incorrect in 1 df
Distance variable(s) is/are incorrect in 2 df
Distance variable(s) is/are incorrect in 2 df
Distance variable(s) is/are incorrect in 2 df
Distance variable(s) is/are incorrect in 2 df
Distance variable(s) is/are incorrect in 2 df
Distance variable(s) is/are incorrect in 2 df
Distance variable(s) is/are incorrect in 2 df
Distance variable(s) is/are incorrect in 3 df
Distance variable(s) is/are incorrect in 4 df


C:\Users\Yevhen\AppData\Roaming\Python\Python311\site-packages\geopy\point.py:472: UserWarning: Latitude normalization has been prohibited in the newer versions of geopy, because the normalized value happened to be on a different pole, which is probably not what was meant. If you pass coordinates as positional args, please make sure that the order is (latitude, longitude) or (y, x) in Cartesian terms.
  return cls(*args)


### 2.6 The same for odometer, but only for those rows who have previous data. 

In [1425]:
#lets'calculate distance between rows for  Odometer
def odometer_distance_for_all_tables (df_list):
    """Takes all dataframes and calculates odometer distance for all of them"""
    def odometer_distance_calc(curr_od, prev_od, index_of_df):
        try:
            #if not(np.isnan(curr_loc) or np.isnan(prev_loc.all())):
            dst = (curr_od - prev_od)*1000 #to get distance in meters
        except ValueError: 
            print(f'Distance variable(s) is/are incorrect in {index_of_df} df')
        return dst
    
    def prev_odometer(df):
        df['prev_od'] = df.sort_values(['id', 'timestamp', 'seq_number']).groupby('id')['odometer_m'].shift()
        return None 
    
    for i, df in enumerate(df_list): 
        prev_odometer(df)
        df['od_distance'] = df.sort_values(['id', 'timestamp']).apply(lambda x: odometer_distance_calc(x['odometer_m'], x['prev_od'], i), axis= 1)
    return None 

In [1426]:
odometer_distance_for_all_tables(fact_tables)

### 2.7 Timedelta (in seconds) between records: 

In [1427]:
#Let's calculate time gaps between records in seconds: 
def timediff_corr_rows(df_list):
    """Takes all dataframes and calculates timedelta between sequence of rows for them"""
    def timediff_curr_prev(timestamp, prev_timestamp, index_of_df):
        try:
            #if not(np.isnan(curr_loc) or np.isnan(prev_loc.all())):
            timediff = pd.Timedelta(timestamp - prev_timestamp).total_seconds() #to get time in seconds
        except ValueError: 
            print(f'Distance variable(s) is/are incorrect in {index_of_df} df')
        return timediff
    
    def prev_timestamp(df):
        df['prev_timestamp'] = df.sort_values(['id', 'timestamp', 'seq_number']).groupby('id')['timestamp'].shift()
        return None 
    
    for i, df in enumerate(df_list): 
        prev_timestamp(df)
        df['timediff'] = df.sort_values(['id', 'timestamp']).apply(lambda x: timediff_curr_prev(x['timestamp'], x['prev_timestamp'], i), axis= 1)
    return None 






In [1428]:
timediff_corr_rows(fact_tables)

In [1429]:
fact_tables[0][['timestamp', 'timediff']]

,timestamp,timediff
0,2024-01-01 15:00:00,NaN
1,2024-01-01 15:04:00,240.0
2,2024-01-01 15:07:00,180.0
3,2024-01-01 15:09:00,120.0
4,2024-01-01 15:12:00,180.0
...,...,...
199,2024-01-02 10:24:00,120.0
200,2024-01-02 10:26:00,120.0
201,2024-01-02 10:27:00,60.0
202,2024-01-02 10:28:00,60.0


### 2.8 And let's created trips. 

In [1430]:
#New Rides maker:

#Definition of the NEW trip = when car is ignited without preceding ignition or with preceding engine-off and spent in this 
#switched off state more than half an hour (or....well....it's discussable). 
trips_timeout = 30*60; #in seconds


def trips(df_list):
    """Takes all dataframes and creates trips division in them"""
    def new_trips_flager(timediff, curr_ign, prev_ign, index_of_df):
        new_trip = 0
        if (timediff > trips_timeout and curr_ign == 1 and prev_ign == 0) or (np.isnan(timediff) and np.isnan(prev_ign) and
                                                                             curr_ign == 1): 
            return 1 
        else: 
            return 0 
#         except ValueError: 
#             print(f'Distance variable(s) is/are incorrect in {index_of_df} df')
    
    def prev_ign_status(df):
        df['prev_fill_ignition'] = df.sort_values(['id', 'timestamp', 'seq_number']).groupby('id')['ignition_filled'].shift()
        return None 
    
    for i, df in enumerate(df_list): 
        prev_ign_status(df)
        df['new_trip'] = df.sort_values(['id', 'timestamp']).apply(lambda x: new_trips_flager(x['timediff'], 
                                                                   x['ignition_filled'], x['prev_fill_ignition'], i), axis= 1)
    return None 


#Rides seq_numbers (and ids for this batches): 
def trips_seq(df_list): 
    for df in df_list: 
        df['trip_id'] = df.sort_values(['id', 'timestamp', 'seq_number']).groupby('id')['new_trip'].cumsum()
    return None

In [1431]:
trips(fact_tables)
trips_seq(fact_tables)

In [1432]:
fact_tables[0].columns

Index(['id', 'timestamp', 'seq_number', 'ignition', 'ignition_filled',
       'odometer_m', 'pos', 'fuel_%', 'prev_pos', 'current_lat', 'current_lon',
       'prev_lat', 'prev_lon', 'current_loc', 'prev_loc', 'gps_distance',
       'prev_od', 'od_distance', 'prev_timestamp', 'timediff',
       'prev_fill_ignition', 'new_trip', 'trip_id'],
      dtype='object')

### 2.9 Creation of resultive table (from all batches) 

In [1433]:
def intermidiate_resultative_table(df_list): 
    common_table = None
    for i, df in enumerate(fact_tables): 
        fact_tables[i]=fact_tables[i][['id', 'trip_id', 'timestamp', 'seq_number', 'timediff',
                                   'ignition_filled', 'odometer_m', 'prev_od', 'od_distance',  'current_loc', 'prev_loc',
                                   'gps_distance', 'fuel_%']]
        fact_tables[i].rename(columns={'seq_number':'event_sec_id', 'timediff': 'timespent', 'ignition_filled':'ignition_state', 
                                  'odometer_m':'curr_od_km', 'prev_od':'prev_od_km' 
                                  }, inplace=True)
        if common_table is None: 
            common_table = fact_tables[i].copy()
        else: 
            common_table = pd.concat([common_table, fact_tables[i]], copy=True)
    return common_table
    
    
    


In [1434]:
common_table = intermidiate_resultative_table(fact_tables)

C:\Users\Yevhen\AppData\Local\Temp\ipykernel_17496\743787015.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fact_tables[i].rename(columns={'seq_number':'event_sec_id', 'timediff': 'timespent', 'ignition_filled':'ignition_state',
C:\Users\Yevhen\AppData\Local\Temp\ipykernel_17496\743787015.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fact_tables[i].rename(columns={'seq_number':'event_sec_id', 'timediff': 'timespent', 'ignition_filled':'ignition_state',
C:\Users\Yevhen\AppData\Local\Temp\ipykernel_17496\743787015.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame


In [1435]:
common_table.columns

Index(['id', 'trip_id', 'timestamp', 'event_sec_id', 'timespent',
       'ignition_state', 'curr_od_km', 'prev_od_km', 'od_distance',
       'current_loc', 'prev_loc', 'gps_distance', 'fuel_%'],
      dtype='object')

In [1436]:
#fact_tables[2]['gps_distance'] = fact_tables[2].sort_values(['id', 'timestamp']).apply(lambda x: distance_calc(x['current_loc'], x['prev_loc']), axis= 1)

In [1437]:
##Need an id for sections in one trip: 
common_table['trip_section'] = common_table.sort_values(['id', 'trip_id','timestamp', 'event_sec_id']).groupby(['id', 'trip_id'])['gps_distance'].cumcount()
common_table.loc[common_table.trip_section==0, 'timespent']=0

In [1438]:
sample = common_table.copy()

In [1439]:
sample

,id,trip_id,timestamp,event_sec_id,timespent,ignition_state,curr_od_km,prev_od_km,od_distance,current_loc,prev_loc,gps_distance,fuel_%,trip_section
0,334455,1,2024-01-01 15:00:00,1,0.0,1,NaN,NaN,NaN,"(53.332025, -6.275705)",NaN,NaN,100.0,0
1,334455,1,2024-01-01 15:04:00,1,240.0,1,NaN,NaN,NaN,"(53.331461, -6.30369)","(53.332025, -6.275705)",1865.458790,NaN,1
2,334455,1,2024-01-01 15:07:00,1,180.0,1,NaN,NaN,NaN,"(53.330896, -6.331675)","(53.331461, -6.30369)",1865.487137,NaN,2
3,334455,1,2024-01-01 15:09:00,1,120.0,1,NaN,NaN,NaN,"(53.330332, -6.35966)","(53.330896, -6.331675)",1865.507988,NaN,3
4,334455,1,2024-01-01 15:12:00,1,180.0,1,NaN,NaN,NaN,"(53.329767, -6.387645)","(53.330332, -6.35966)",1865.536334,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725,334459,3,2024-01-07 19:02:53,1,69.0,1,5107.81,5105.95,1860.0,"(50.145249, 8.641091)","(50.15498, 8.619314)",1895.818315,51.0,196
726,334459,3,2024-01-07 19:03:34,1,41.0,1,5109.67,5107.81,1860.0,"(50.135518, 8.662869)","(50.145249, 8.641091)",1896.135262,51.0,197
727,334459,3,2024-01-07 19:04:39,1,65.0,1,5111.53,5109.67,1860.0,"(50.125787, 8.684646)","(50.135518, 8.662869)",1896.334799,51.0,198
728,334459,3,2024-01-07 19:05:30,1,51.0,1,5113.39,5111.53,1860.0,"(50.116055, 8.706423)","(50.125787, 8.684646)",1896.656507,50.0,199


In [1440]:
sample

,id,trip_id,timestamp,event_sec_id,timespent,ignition_state,curr_od_km,prev_od_km,od_distance,current_loc,prev_loc,gps_distance,fuel_%,trip_section
0,334455,1,2024-01-01 15:00:00,1,0.0,1,NaN,NaN,NaN,"(53.332025, -6.275705)",NaN,NaN,100.0,0
1,334455,1,2024-01-01 15:04:00,1,240.0,1,NaN,NaN,NaN,"(53.331461, -6.30369)","(53.332025, -6.275705)",1865.458790,NaN,1
2,334455,1,2024-01-01 15:07:00,1,180.0,1,NaN,NaN,NaN,"(53.330896, -6.331675)","(53.331461, -6.30369)",1865.487137,NaN,2
3,334455,1,2024-01-01 15:09:00,1,120.0,1,NaN,NaN,NaN,"(53.330332, -6.35966)","(53.330896, -6.331675)",1865.507988,NaN,3
4,334455,1,2024-01-01 15:12:00,1,180.0,1,NaN,NaN,NaN,"(53.329767, -6.387645)","(53.330332, -6.35966)",1865.536334,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725,334459,3,2024-01-07 19:02:53,1,69.0,1,5107.81,5105.95,1860.0,"(50.145249, 8.641091)","(50.15498, 8.619314)",1895.818315,51.0,196
726,334459,3,2024-01-07 19:03:34,1,41.0,1,5109.67,5107.81,1860.0,"(50.135518, 8.662869)","(50.145249, 8.641091)",1896.135262,51.0,197
727,334459,3,2024-01-07 19:04:39,1,65.0,1,5111.53,5109.67,1860.0,"(50.125787, 8.684646)","(50.135518, 8.662869)",1896.334799,51.0,198
728,334459,3,2024-01-07 19:05:30,1,51.0,1,5113.39,5111.53,1860.0,"(50.116055, 8.706423)","(50.125787, 8.684646)",1896.656507,50.0,199


### 2.10 Extracting distance covered from odometer, even if it reports only once per long period

In [1505]:
#Function to set distance, find speed and find

def speed_and_distance(df_input): 
    """Function to extract corrected distance and to find speed (to make checks)"""
    
    speed_limit = 250_000/3600 #let's set some speed, over which we don't believe over vehiecles can go. 
    
    def distance_choicer(df_row):
        "To choose distance according to conditions"
        dist = 0 
        if df_row['odometer_valuelability'] and df_row['od_distance']>=0 and not(np.isnan(df_row['od_distance'])):
            #and df_row['od_speed'] <= speed_limit:
                dist = df_row['od_distance']
        elif df_row['gps_distance']>=0 and df_row['gps_speed']>=0 and  not(np.isnan(df_row['gps_distance'])):
              #and df_row['gps_speed']<= speed_limit:
                dist = df_row['gps_distance']
        return dist

    def speed_choicer(df_row):
        "To find speed according to rules"
        spd = 0 
        if df_row['odometer_valuelability'] and df_row['od_distance']>=0 and not(np.isnan(df_row['od_speed']))\
            and df_row['od_speed'] <= speed_limit:
                spd = df_row['od_speed']
        elif df_row['gps_distance']>=0 and df_row['gps_speed']>=0 and  not(np.isnan(df_row['gps_speed']))\
              and df_row['gps_speed']<= speed_limit:
                spd = df_row['gps_speed']
        return spd
    
    df = df_input.copy()
    groupby= df.sort_values(['id', 'trip_id','timestamp', 'event_sec_id']).groupby(['id', 'trip_id'], as_index=False)\
                ['curr_od_km']
    first_val=groupby.first()
    min_val = groupby.min()


    min_val=min_val.merge(first_val, on=['id', 'trip_id'])
    min_val['odometer_valuelability'] = min_val['curr_od_km_x'] == min_val['curr_od_km_y']
    df['trip_section'] = df.sort_values(['id', 'trip_id','timestamp', 'event_sec_id']).groupby(['id', 'trip_id'])['gps_distance'].cumcount()
    #time_commulative to find avg speed then. 
    
    min_val=min_val[['id', 'trip_id', 'odometer_valuelability']]

    df=df.merge(min_val, on = ['id', 'trip_id'], how='left')
    #df['trip_section'] = df.sort_values(['id', 'trip_id','timestamp', 'event_sec_id']).groupby(['id', 'trip_id'])['gps_distance'].cumcount()

    
    
    df['is_odometer'] = df.apply(lambda x:0 if np.isnan(x['curr_od_km']) or x['ignition_state']==0 else 1 , axis= 1)\
    #     .groupby(['id', 'trip_id']).cumsum()
    df['odometer_group']=df.groupby(['id', 'trip_id'])['is_odometer'].cumsum()
    
    od_values=df.groupby(['id', 'trip_id', 'odometer_group'], as_index=False)['curr_od_km'].max() 
    od_values=od_values.loc[od_values['odometer_group'] > 0]
    od_values['od_distance']=od_values.sort_values(['id','trip_id','odometer_group']).groupby(['id', 'trip_id'])['curr_od_km']\
                                .transform(lambda x:( x - x.shift(1))*1000)# ['od_diff']
    #shift(-1)
    timecum = df.loc[(df['trip_section']>0)&(df['is_odometer']!= 1)&(df['odometer_group']!=0)]\
                .groupby(['id', 'trip_id', 'odometer_group'], as_index=False)['timespent'].sum()
    timeFirst = df.loc[(df['trip_section']!=0)&(df['is_odometer']== 1)&(df['odometer_group']!=0)]\
                .groupby(['id', 'trip_id', 'odometer_group'], as_index=False)['timespent'].max()
    timeFirst.rename(columns = {'timespent':'quant_time'}, inplace=True)
    timeFirst=timeFirst.merge(timecum, on=['id', 'trip_id', 'odometer_group'], how='outer').fillna(0)
    timeFirst.sort_values(['id', 'trip_id', 'odometer_group'], inplace=True)
    timeFirst['prev_timecum']=timeFirst.sort_values(['id', 'trip_id', 'odometer_group']).groupby(['id', 'trip_id'])['timespent'].shift(1)
    timeFirst['ttr']= timeFirst.quant_time+timeFirst.prev_timecum
    timeFirst= timeFirst[['id', 'trip_id', 'odometer_group', 'ttr']]
    
    od_values=od_values.merge(timecum, on = ['id', 'trip_id', 'odometer_group'], how = 'left')
    od_values=od_values.merge(timeFirst, on=['id', 'trip_id', 'odometer_group'])

    od_values['avg_od_speed']=od_values['od_distance']/od_values['ttr']
    od_values['avg_od_speed']=od_values['avg_od_speed'].apply(lambda x: np.nan if x<0 else x)
    od_values['avg_od_speed']=od_values['avg_od_speed'].shift(-1)
    od_values= od_values[['id', 'trip_id', 'odometer_group', 'avg_od_speed']]
    od_values.drop_duplicates(subset=['id', 'trip_id', 'odometer_group'], inplace=True)
    df=df.merge(od_values, on=['id','trip_id', 'odometer_group'], how = 'left')
    
    
    #shifting speeds
    s=df.loc[(df.is_odometer==1)]['avg_od_speed'].shift(1)
    df=df.join(s, how='left', rsuffix='_2')
    df['avg_od_speed']= df.apply(lambda x: x['avg_od_speed'] if x['is_odometer']==0 else x['avg_od_speed_2'] , axis=1)
    #speeds for gps and odemeter
    df['od_speed']=df['od_distance']/df['timespent']
    df['gps_speed']=df['gps_distance']/df['timespent']
    
    df.loc[df.trip_section==0, ['gps_distance']]=0
    df.loc[df.trip_section==0, ['od_distance']]=0
    df.loc[df.trip_section==0, ['avg_od_speed']]=0
    df.loc[df.trip_section==0, ['od_speed']]=0
    df.loc[df.trip_section==0, ['gps_speed']]=0
    
   


    df['distance'] = df.apply(distance_choicer, axis=1)
    df['speed'] = df.apply(speed_choicer, axis= 1)
    df=df[['id', 'trip_id', 'timestamp', 'event_sec_id', 'timespent',
       'ignition_state', 'curr_od_km', 'prev_od_km', 'od_distance',
       'current_loc', 'prev_loc', 'gps_distance', 'fuel_%', 'trip_section',
       'odometer_valuelability', 'is_odometer', 'odometer_group',
       'avg_od_speed', 'avg_od_speed_2', 'od_speed', 'gps_speed', 'distance',
       'speed']]
    df.loc[(df.gps_speed>speed_limit) & (df.od_speed>speed_limit),'avg_od_speed']= 0
    df.loc[(df.gps_speed>speed_limit) & (df.od_speed>speed_limit),'avg_od_speed_2'] = 0
    df.loc[(df.gps_speed>speed_limit) & (df.od_speed>speed_limit),'od_speed'] = 0
    df.loc[(df.gps_speed>speed_limit) & (df.od_speed>speed_limit),'gps_speed'] = 0
    #df.loc[(df.gps_speed>speed_limit) & (df.od_speed>speed_limit),'distance'] = 0 - no need. Sensor probably can send data 
    #like sequence of batches, doesn't mean vehicle covered this distance in few secs. 
    df.loc[(df.gps_speed>speed_limit) & (df.od_speed>speed_limit),'speed'] = 0


    df.drop(df.loc[df.event_sec_id > 1].index, inplace=True)
    df = df[['id', 'trip_id', 'trip_section', 'timestamp', 'timespent', 
                 'current_loc', 'prev_loc', 'fuel_%', 'speed', 'distance', 'avg_od_speed', 'gps_speed']]
    return df

In [1506]:
sample1=speed_and_distance(common_table)

### 2.11 Clearing fuel data

In [1507]:
def fuel_cleaning(df_input):
    """To clear fuel change in table"""
    
    def fuel_percentage(df_row):
        "To put avg fuel change in table"
        res= np.nan
        if df_row['is_fuel'] == 1:
            res=df_row['fuel_%']
        elif df_row['fuel_%_per_timespent'] >=0: 
            res=(-1)*df_row['timespent']*df_row['fuel_%_per_timespent']
        return res

    df=df_input.copy()
    df['is_fuel'] =df.apply(lambda x:0 if np.isnan(x['fuel_%']) else 1, axis= 1)
    df['fuel_seq']=df.sort_values(['id', 'trip_id', 'trip_section'])\
                            .groupby(['id', 'trip_id'])['is_fuel'].cumsum()
    df['fuel_%'] = df.sort_values(['id', 'trip_id', 'timestamp']).groupby(['id', 'trip_id'])['fuel_%']\
            .fillna(method='ffill')
    df_groupby = df.groupby(['id', 'trip_id', 'fuel_seq', 'fuel_%'], as_index=False).agg({ 'trip_section': 'nunique', 
                                                                                    'distance': 'sum', 
                                                                                    'timespent': 'sum'
                                                                             })
    df_groupby['delta_fuel'] = df_groupby.sort_values(['id', 'trip_id', 'fuel_seq'])\
                                .groupby(['id', 'trip_id'])['fuel_%'].transform(lambda x: x - x.shift(-1))
    
    df_groupby['fuel_%_per_timespent'] = df_groupby.delta_fuel/df_groupby.timespent
    df_groupby['fuel_%_per_distance'] = df_groupby.delta_fuel/df_groupby.distance
    df_groupby= df_groupby[['id', 'trip_id', 'fuel_seq', 'delta_fuel', 'fuel_%_per_distance', 'fuel_%_per_timespent']]
    df=df.merge(df_groupby, on=['id', 'trip_id', 'fuel_seq'], how='left')
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    
    #Calculate fuel percents for many rows where there is none
    df['fuel_perc_change']= df.apply(fuel_percentage, axis=1)
    
    #Just to fill in the table (forward+some calculation)

    df['fuel_perc_change']=df.sort_values(['id', 'trip_id', 'timestamp', 'fuel_seq'])\
                                .groupby(['id', 'trip_id', 'fuel_seq'])['fuel_perc_change'].cumsum()
    df['fuel_spent%_per_time_spent']= df['fuel_%_per_timespent'].apply(lambda x: x if x>=0 else np.nan)
    df['fuel_spent%_per_time_spent']= df.groupby('id')['fuel_spent%_per_time_spent'].fillna(method='ffill')
    df['spent_fuel%'] = df['fuel_spent%_per_time_spent']*df['timespent']
    df['spent_fuel%']=df['spent_fuel%'].shift(1)
    df['spent_fuel%']=df['spent_fuel%'].fillna(0)
    df['delta_fuel']=df['delta_fuel'].fillna(0)
    #df.replace([np.inf, -np.inf], np.nan, inplace=True)
    
    return df
    

In [1578]:
test = fuel_cleaning(sample1)

proved_fuel_spent = test.loc[test.delta_fuel >= 0].groupby(['id', 'trip_id', 'delta_fuel', 'fuel_seq'], \
                                                                  as_index=False)['fuel_perc_change'].count()
proved_fuel_spent= proved_fuel_spent.groupby(['id', 'trip_id'], as_index=False)['delta_fuel'].sum()

proved_fuel_refill =  test.loc[test.delta_fuel < 0].groupby(['id', 'trip_id', 'delta_fuel', 'fuel_seq'], as_index=False)['fuel_perc_change'].count()
proved_fuel_refill =  proved_fuel_refill.groupby(['id', 'trip_id'], as_index=False).agg({'delta_fuel': 'sum', 'fuel_seq': 'count'})
proved_fuel_refill['delta_fuel'] = proved_fuel_refill['delta_fuel']*-1

In [1579]:
proved_fuel_refill

,id,trip_id,delta_fuel,fuel_seq
0,334455,1,60.0,1
1,334456,1,148.0,5
2,334457,1,60.0,1
3,334457,2,188.0,2
4,334458,1,60.0,1
5,334459,2,81.0,1


In [1580]:
test['first_location'] = test.groupby(['id', 'trip_id'])['current_loc'].transform('first')

In [1581]:
def distance_between_first_and_last(df):
    dst = 0
    try:
        dst = distance.distance(df['current_loc'] , df['first_location']).m
    except ValueError: 
        print("GeoData isn't correct")
    return dst 

In [1582]:
test['max_distance_a'] = test.apply(distance_between_first_and_last, axis=1)

C:\Users\Yevhen\AppData\Roaming\Python\Python311\site-packages\geopy\point.py:472: UserWarning: Latitude normalization has been prohibited in the newer versions of geopy, because the normalized value happened to be on a different pole, which is probably not what was meant. If you pass coordinates as positional args, please make sure that the order is (latitude, longitude) or (y, x) in Cartesian terms.
  return cls(*args)


GeoData isn't correct
GeoData isn't correct
GeoData isn't correct


In [1583]:
farpoints = test.groupby(['id', 'trip_id'], as_index=False)['max_distance_a'].max()
farpoints = farpoints.merge(test, left_on=['id', 'trip_id', 'max_distance_a'], 
                             right_on=['id', 'trip_id', 'max_distance_a'])[['id', 'trip_id', 
                                                                            'max_distance_a', 'current_loc', 'timestamp']]
farpoints.drop_duplicates(subset = ['id', 'trip_id', 'max_distance_a'], inplace=True)


In [1584]:
last_points = test.groupby(['id', 'trip_id'], as_index=False)['timestamp'].max()
last_points = last_points.merge(test, left_on=['id', 'trip_id', 'timestamp'], 
                             right_on=['id', 'trip_id', 'timestamp'])[['id', 'trip_id', 
                                                                       'max_distance_a', 'current_loc', 'timestamp']]


In [1585]:
last_points.rename(columns = {'max_distance_a': 'distination_distance_m', 'current_loc':'destination', 'timestamp': 'finish'},
                  inplace=True)
farpoints.rename(columns={'max_distance_a': 'farest_point_m', 'current_loc':'farest_geo', 'timestamp': 'time_reached'}, 
                inplace=True)

In [1586]:
points=farpoints.merge(last_points, on=['id', 'trip_id'])
points.drop_duplicates(subset=['id','trip_id'], inplace = True)

In [1587]:
test= test[['id', 'trip_id', 'timestamp', 'timespent', 'speed', 'distance',
           'avg_od_speed', 'gps_speed', 'spent_fuel%', 'first_location']]

test.columns = ['id', 'trip_id', 'timestamp', 'timespent_s', 'speed_m_s', 'distance_m', 'avg_od_speed_m_s', 'gps_speed_m_s', 
               'spent_fuel%', 'first_location']

In [1588]:
test['timespent_s']=test['timespent_s'].fillna(0)
test['timespent_s']=test['timespent_s'].apply(int)
test['speed_m_s'] = test['speed_m_s'].apply(lambda x: round(x, 2))
test['distance_m']  = test['distance_m'].apply(int)
test['avg_od_speed_m_s'] = test['avg_od_speed_m_s'].apply(lambda x: round(x, 2))
test['gps_speed_m_s'] = test['gps_speed_m_s'].apply(lambda x: round(x,2))
test['spent_fuel%'] = test['spent_fuel%'].apply(lambda x: round(x,2))

In [1589]:
##This table is ready. 
test.to_csv('Outputs/final_table_of_facts.csv')

In [1590]:
##Dimension tables as well: 
accounts.to_csv('Outputs/accounts_clear.csv')
assets.to_csv('Outputs/assets_clear.csv')

## Final! Aggregation: 

In [1682]:
##Trips level_first: 
trips_table=test.groupby(['id', 'trip_id'], as_index=False).agg(start_trip=('timestamp', 'min'),  end_trip=('timestamp', 'max'), 
                                                duration_s=('timespent_s', 'sum'), total_distance_m=('distance_m', 'sum'), 
                                                median_od_speed_m_s=('avg_od_speed_m_s', 'median'), 
                                                median_gps_speed_m_s=('gps_speed_m_s', 'median'),
                                                cumulative_fuel_spent_assump_pc=('spent_fuel%', 'sum'), 
                                                starting_point=('first_location', 'min')
                                          )

In [1683]:
trips_table['start_date']=trips_table['start_trip'].dt.date

In [1684]:
trips_table=trips_table.merge(proved_fuel_spent, on=['id', 'trip_id'], how='left')
trips_table.rename(columns={'delta_fuel':'fuel_spent_%'}, inplace=True)

In [1685]:
trips_table=trips_table.merge(proved_fuel_refill, on=['id', 'trip_id'], how='left')

In [1686]:
trips_table[['fuel_spent_%','delta_fuel','fuel_seq']]=trips_table[['fuel_spent_%','delta_fuel','fuel_seq']].fillna(0)
trips_table['fuel_spent_%']=trips_table['fuel_spent_%'].apply(int)
trips_table['delta_fuel']= trips_table['delta_fuel'].apply(int)
trips_table['fuel_seq']=trips_table['fuel_seq'].apply(int)

In [1687]:
#quick check of 4 refills during trip #1 with id = 334456

In [1688]:
check=fuel_cleaning(sample1)
check = check.loc[(check.id == 334456) &(check.trip_id==1)]
check = check[['id', 'trip_id', 'timestamp', 'fuel_%']]

In [1689]:
check['refill']=check.sort_values('timestamp')['fuel_%'].transform(lambda x: x - x.shift(1))

In [1690]:
check.loc[check.refill>0]#okaaaay, so 3 times it was a change by 1%. Let's manually (is this still script though? But just once doesn't count...) just fix that.

,id,trip_id,timestamp,fuel_%,refill
282,334456,1,2024-01-04 11:58:27,90.0,69.0
311,334456,1,2024-01-04 12:37:27,63.0,1.0
325,334456,1,2024-01-04 12:53:42,51.0,1.0
353,334456,1,2024-01-04 14:30:00,100.0,76.0
419,334456,1,2024-01-04 15:31:38,36.0,1.0


In [1691]:
trips_table.loc[(trips_table.id == 334456) & (trips_table.trip_id)==1, 'fuel_seq'] = trips_table.loc[(trips_table.id == 334456) & (trips_table.trip_id)==1, 'fuel_seq'] -3
trips_table.loc[(trips_table.id == 334456) & (trips_table.trip_id)==1, 'delta_fuel'] = trips_table.loc[(trips_table.id == 334456) & (trips_table.trip_id)==1, 'delta_fuel'] -3

In [1692]:
points

,id,trip_id,farest_point_m,farest_geo,time_reached,distination_distance_m,destination,finish
0,334455,1,186656.627296,"(53.275578, -9.074217)",2024-01-01 19:53:00,186656.627296,"(53.275578, -9.074217)",2024-01-01 19:53:00
1,334455,2,186656.627296,"(53.332025, -6.275705)",2024-01-02 10:28:00,186656.627296,"(53.332025, -6.275705)",2024-01-02 10:28:00
2,334456,1,222725.499354,"(52.241316, -7.161516)",2024-01-04 13:31:03,0.000000,"(54.098194, -8.4042)",2024-01-04 16:02:11
3,334457,1,252319.167553,"(51.883981, -9.582074)",2024-01-01 14:31:32,252319.167553,"(51.883981, -9.582074)",2024-01-01 14:31:32
4,334457,2,230383.566381,"(54.198514, -9.925246)",2024-01-02 14:21:48,0.000000,"(53.969733, -6.426101)",2024-01-02 17:02:16
5,334458,1,235464.401183,"(52.473127, -1.852234)",2024-01-01 19:53:00,235464.401183,"(52.473127, -1.852234)",2024-01-01 19:53:00
6,334458,2,235464.401183,"(54.555905, -1.228622)",2024-01-02 10:28:00,160616.400501,"(51.500926, -0.123827)",2024-01-02 12:08:36
7,334459,1,374115.015207,"(50.106324, 8.7282)",2024-01-05 15:28:48,374115.015207,"(50.106324, 8.7282)",2024-01-05 15:28:48
8,334459,2,374115.015207,"(52.052579, 4.372771)",2024-01-06 18:05:37,374115.015207,"(52.052579, 4.372771)",2024-01-06 18:05:37
9,334459,3,374115.015207,"(50.106324, 8.7282)",2024-01-07 19:05:57,374115.015207,"(50.106324, 8.7282)",2024-01-07 19:05:57


In [1693]:
trips_table.rename(columns={'delta_fuel': 'refueled_%', 'fuel_seq':'refuiled_times'}, inplace=True)
trips_table = trips_table.merge(points, on =['id', 'trip_id'], how='left' )[['id', 'trip_id', 'start_trip', 'end_trip', 'duration_s',
       'total_distance_m', 'median_od_speed_m_s', 'median_gps_speed_m_s',
       'cumulative_fuel_spent_assump_pc', 'starting_point', 'start_date',
       'fuel_spent_%', 'refueled_%', 'refuiled_times', 'farest_point_m', 'farest_geo', 'time_reached',
       'distination_distance_m', 'destination' ]]

In [1694]:
trips_table.columns

Index(['id', 'trip_id', 'start_trip', 'end_trip', 'duration_s',
       'total_distance_m', 'median_od_speed_m_s', 'median_gps_speed_m_s',
       'cumulative_fuel_spent_assump_pc', 'starting_point', 'start_date',
       'fuel_spent_%', 'refueled_%', 'refuiled_times', 'farest_point_m',
       'farest_geo', 'time_reached', 'distination_distance_m', 'destination'],
      dtype='object')

In [1695]:
###Let's convert some types
#m/s to km/h
kmh= 3600/1000
km = 1000

trips_table['duration'] = trips_table.duration_s.apply(lambda x: str(timedelta(seconds=x)))
trips_table.total_distance_m = trips_table.total_distance_m / km 
trips_table.total_distance_m = trips_table.total_distance_m.apply(math.floor)
trips_table.total_distance_m = trips_table.total_distance_m.apply(int)
trips_table.median_od_speed_m_s = trips_table.median_od_speed_m_s * kmh
trips_table.median_gps_speed_m_s = trips_table.median_gps_speed_m_s * kmh
trips_table.farest_point_m = (trips_table.farest_point_m / km)
trips_table.distination_distance_m = (trips_table.distination_distance_m / km)
trips_table.farest_point_m = trips_table.farest_point_m.apply(math.floor)
trips_table.distination_distance_m = trips_table.distination_distance_m.apply(math.floor)
trips_table.farest_point_m = trips_table.farest_point_m.apply(int)
trips_table.distination_distance_m = trips_table.distination_distance_m.apply(int)
trips_table.median_od_speed_m_s = trips_table.median_od_speed_m_s.apply(int)
trips_table.median_gps_speed_m_s = trips_table.median_gps_speed_m_s.apply(int)


In [1697]:
trips_table.columns

Index(['id', 'trip_id', 'start_trip', 'end_trip', 'duration_s',
       'total_distance_m', 'median_od_speed_m_s', 'median_gps_speed_m_s',
       'cumulative_fuel_spent_assump_pc', 'starting_point', 'start_date',
       'fuel_spent_%', 'refueled_%', 'refuiled_times', 'farest_point_m',
       'farest_geo', 'time_reached', 'distination_distance_m', 'destination',
       'duration'],
      dtype='object')

In [1698]:
trips_table.columns= ['id', 'trip_id', 'start_time', 'end_time', 'duration_s', 'total_distance_km', 'median_odometer_speed',
                     'median_gps_speed', 'fuel_spent_calc_%', 'start_point', 'start_date', 'fuel_spent_%', 'refueled_%', 
                      'refuiled_times', 'farest_point', 'farest_geo', 'time_reached', 'destination_distance', 'destination',
                      'duration']
trips_table['C'] = trips_table.apply(lambda x: x['destination'] if x['destination_distance']<x['farest_point']\
                                     else np.nan, axis=1)

In [1699]:
def city_by_geo(df_col):
    try: 
        geolocator = Nominatim(user_agent="extractorAPI")
        location = geolocator.reverse(df_col)
        geo= location.raw['address'].get('country','')+' || '+ location.raw['address'].get('city', '')
        return geo
    except: 
        print('Not a location')



trips_table['A_geo'] = trips_table['start_point'].apply(city_by_geo)
trips_table['B_geo'] = trips_table['farest_geo'].apply(city_by_geo)
trips_table['C_geo'] = trips_table['C'].apply(city_by_geo)


Not a location
Not a location
Not a location
Not a location
Not a location
Not a location
Not a location


In [1700]:
trips_table= trips_table[['id','trip_id', 'start_date','start_time', 'end_time','duration','duration_s','A_geo',
       'B_geo', 'C_geo', 'start_point', 'farest_geo',  'C', 'total_distance_km', 'median_odometer_speed', 'median_gps_speed',
       'fuel_spent_calc_%','fuel_spent_%','refueled_%', 'refuiled_times']]

trips_table.rename(columns={'farest_geo' : 'B', 'start_point': 'A'}, inplace=True)

C:\Users\Yevhen\AppData\Local\Temp\ipykernel_17496\3421971787.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trips_table.rename(columns={'farest_geo' : 'B', 'start_point': 'A'}, inplace=True)


In [1701]:
trips_long=trips_table.merge(assets, left_on=['id'], right_on=['data_source']).copy()
trips_long=trips_long.merge(accounts, on = ['account_id'])

In [1702]:
trips_long=trips_long[['account_name', 'license_plate', 'maker_fixed', 'model', 'tank_capacity','trip_id', 'start_date','start_time', 'end_time','duration', 'duration_s','A_geo',
       'B_geo', 'C_geo', 'A', 'B',  'C', 'total_distance_km', 'median_odometer_speed', 'median_gps_speed',
       'fuel_spent_calc_%','fuel_spent_%','refueled_%', 'refuiled_times' ]]

In [1703]:
trips_long['fuel_spent_calc_%']=trips_long['fuel_spent_calc_%']*trips_long['tank_capacity']/100
trips_long['fuel_spent_%']=trips_long['fuel_spent_%']*trips_long['tank_capacity']/100
trips_long['refueled_%']=trips_long['refueled_%']*trips_long['tank_capacity']/100

trips_long.rename(columns = {'fuel_spent_calc_%': 'calc_fuel_spent', 'fuel_spent_%':'fact_fuel_spent',\
                             'refueled_%': 'refueld', 'total_distance_km':'total_distance'}, inplace=True)

In [1704]:
trips_long.to_csv('Outputs/trips_table.csv')

In [1705]:
trips_long.columns

Index(['account_name', 'license_plate', 'maker_fixed', 'model',
       'tank_capacity', 'trip_id', 'start_date', 'start_time', 'end_time',
       'duration', 'duration_s', 'A_geo', 'B_geo', 'C_geo', 'A', 'B', 'C',
       'total_distance', 'median_odometer_speed', 'median_gps_speed',
       'calc_fuel_spent', 'fact_fuel_spent', 'refueld', 'refuiled_times'],
      dtype='object')

In [1706]:
####WOOOW, as there is no trips started in one day and lasted till another, I'll use this table mostly for aggregation 
daily_agg = trips_long.groupby(['start_date', 'account_name', 'license_plate', 'maker_fixed', 'model'], as_index=False).agg(
                                                trips=('trip_id', 'count'),  duration = ('duration', 'sum'),
                                                duration_s=('duration_s', 'sum'), 
                                                total_distance=('total_distance', 'sum'), 
                                                calc_fuel_spent =('calc_fuel_spent', 'sum'), 
                                                fact_fuel_spent=('fact_fuel_spent', 'sum'),
                                                refueld=('refueld', 'sum'), refuiled_times=('refuiled_times', 'sum')
)

In [1707]:
daily_agg.to_csv('Outputs/daily_agg.csv')

In [1062]:
df.groupby([df['Date_Time'].dt.date]).mean()

,id,trip_id,timestamp,timespent_s,speed_m_s,distance_m,avg_od_speed_m_s,gps_speed_m_s,spent_fuel%,first_location
0,334455,1,2024-01-01 15:00:00,0,0.00,0,0.00,0.00,0.00,"(53.332025, -6.275705)"
1,334455,1,2024-01-01 15:04:00,240,7.77,1865,NaN,7.77,0.00,"(53.332025, -6.275705)"
2,334455,1,2024-01-01 15:07:00,180,10.36,1865,NaN,10.36,1.03,"(53.332025, -6.275705)"
3,334455,1,2024-01-01 15:09:00,120,15.55,1865,NaN,15.55,0.77,"(53.332025, -6.275705)"
4,334455,1,2024-01-01 15:12:00,180,10.36,1865,NaN,10.36,0.51,"(53.332025, -6.275705)"
...,...,...,...,...,...,...,...,...,...,...
1969,334459,3,2024-01-07 19:02:53,69,26.96,1859,26.96,27.48,0.00,"(52.052579, 4.372771)"
1970,334459,3,2024-01-07 19:03:34,41,45.37,1859,45.37,46.25,0.00,"(52.052579, 4.372771)"
1971,334459,3,2024-01-07 19:04:39,65,28.62,1859,28.62,29.17,0.00,"(52.052579, 4.372771)"
1972,334459,3,2024-01-07 19:05:30,51,36.47,1859,36.47,37.19,1.00,"(52.052579, 4.372771)"


In [872]:
groupby= sample.sort_values(['id', 'trip_id','timestamp', 'event_sec_id']).groupby(['id', 'trip_id'], as_index=False)\
                ['curr_od_km']
#

# grouped = df.groupby('A')
first_val=groupby.first()
min_val = groupby.min()

print(first_val)
print(min_val)

min_val=min_val.merge(first_val, on=['id', 'trip_id'])
min_val['odometer_valuelability'] = min_val['curr_od_km_x'] == min_val['curr_od_km_y']


# DataFrame.merge
# DataFrame.merge(right, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=None, indicator=False, validate=None)[source]


       id  trip_id  curr_od_km
0  334455        1       15.00
1  334455        2      149.00
2  334456        1        1.48
3  334457        1        1.25
4  334457        2      253.08
5  334458        1       19.00
6  334458        2      235.00
7  334459        1     4000.00
8  334459        2     4371.25
9  334459        3     4743.25
       id  trip_id  curr_od_km
0  334455        1        0.00
1  334455        2        2.00
2  334456        1        1.48
3  334457        1        1.25
4  334457        2        0.00
5  334458        1       19.00
6  334458        2      235.00
7  334459        1     4000.00
8  334459        2     4371.25
9  334459        3     4743.25


In [873]:
sample['trip_section'] = sample.sort_values(['id', 'trip_id','timestamp', 'event_sec_id']).groupby(['id', 'trip_id'])['gps_distance'].cumcount()

In [874]:
min_val=min_val[['id', 'trip_id', 'odometer_valuelability']]

sample=sample.merge(min_val, on = ['id', 'trip_id'])
#sample['trip_section'] = sample.sort_values(['id', 'trip_id','timestamp', 'event_sec_id']).groupby(['id', 'trip_id'])['gps_distance'].cumcount()


In [875]:
sample

,id,trip_id,timestamp,event_sec_id,timespent,ignition_state,curr_od_km,prev_od_km,od_distance,current_loc,prev_loc,gps_distance,fuel_%,trip_section,odometer_valuelability
0,334455,1,2024-01-01 15:00:00,1,NaN,1,NaN,NaN,NaN,"(53.332025, -6.275705)",NaN,NaN,100.0,0,False
1,334455,1,2024-01-01 15:04:00,1,240.0,1,NaN,NaN,NaN,"(53.331461, -6.30369)","(53.332025, -6.275705)",1865.458790,NaN,1,False
2,334455,1,2024-01-01 15:07:00,1,180.0,1,NaN,NaN,NaN,"(53.330896, -6.331675)","(53.331461, -6.30369)",1865.487137,NaN,2,False
3,334455,1,2024-01-01 15:09:00,1,120.0,1,NaN,NaN,NaN,"(53.330332, -6.35966)","(53.330896, -6.331675)",1865.507988,NaN,3,False
4,334455,1,2024-01-01 15:12:00,1,180.0,1,NaN,NaN,NaN,"(53.329767, -6.387645)","(53.330332, -6.35966)",1865.536334,NaN,4,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016,334459,3,2024-01-07 19:02:53,1,69.0,1,5107.81,5105.95,1860.0,"(50.145249, 8.641091)","(50.15498, 8.619314)",1895.818315,51.0,196,True
2017,334459,3,2024-01-07 19:03:34,1,41.0,1,5109.67,5107.81,1860.0,"(50.135518, 8.662869)","(50.145249, 8.641091)",1896.135262,51.0,197,True
2018,334459,3,2024-01-07 19:04:39,1,65.0,1,5111.53,5109.67,1860.0,"(50.125787, 8.684646)","(50.135518, 8.662869)",1896.334799,51.0,198,True
2019,334459,3,2024-01-07 19:05:30,1,51.0,1,5113.39,5111.53,1860.0,"(50.116055, 8.706423)","(50.125787, 8.684646)",1896.656507,50.0,199,True


In [876]:
# sample.sort_values(['id', 'trip_id','timestamp', 'event_sec_id']).groupby(['id', 'trip_id'], as_index=False)\
#                 ['curr_od_km'].transform(lambda x: 0 if np.isnan(x) else 1)


sample['is_odometer'] = sample.apply(lambda x:0 if np.isnan(x['curr_od_km']) or x['ignition_state']==0 else 1 , axis= 1)\
#     .groupby(['id', 'trip_id']).cumsum()
sample['odometer_group']=sample.groupby(['id', 'trip_id'])['is_odometer'].cumsum()

In [877]:
od_values=sample.groupby(['id', 'trip_id', 'odometer_group'], as_index=False)['curr_od_km'].max() 
od_values=od_values.loc[od_values['odometer_group'] > 0]

od_values['od_distance']= od_values.sort_values(['id','trip_id','odometer_group']).groupby(['id', 'trip_id'])['curr_od_km']\
    .transform(lambda x:( x - x.shift(1))*1000)# ['od_diff']
#shift(-1)

In [878]:
sample.loc[sample['odometer_group']==1]

,id,trip_id,timestamp,event_sec_id,timespent,ignition_state,curr_od_km,prev_od_km,od_distance,current_loc,prev_loc,gps_distance,fuel_%,trip_section,odometer_valuelability,is_odometer,odometer_group
7,334455,1,2024-01-01 15:19:00,1,120.0,1,15.00,NaN,NaN,"(53.328074, -6.471601)","(53.328638, -6.443616)",1865.606380,NaN,7,False,1,1
8,334455,1,2024-01-01 15:24:00,1,300.0,1,NaN,15.00,NaN,"(53.327509, -6.499586)","(53.328074, -6.471601)",1865.634726,NaN,8,False,0,1
102,334455,2,2024-01-02 08:00:00,1,43620.0,1,149.00,NaN,NaN,"(53.275578, -9.074217)","(53.275578, -9.074217)",0.000000,NaN,0,False,1,1
103,334455,2,2024-01-02 08:01:00,1,60.0,1,NaN,149.00,NaN,"(53.276142, -9.046232)","(53.275578, -9.074217)",1867.893075,NaN,1,False,0,1
104,334455,2,2024-01-02 08:03:00,1,120.0,1,NaN,NaN,NaN,"(53.276707, -9.018247)","(53.276142, -9.046232)",1867.872237,NaN,2,False,0,1
105,334455,2,2024-01-02 08:04:00,1,60.0,1,NaN,NaN,NaN,"(53.277271, -8.990262)","(53.276707, -9.018247)",1867.843913,NaN,3,False,0,1
106,334455,2,2024-01-02 08:05:00,1,60.0,1,NaN,NaN,NaN,"(53.277836, -8.962277)","(53.277271, -8.990262)",1867.823075,NaN,4,False,0,1
107,334455,2,2024-01-02 08:07:00,1,120.0,1,NaN,NaN,NaN,"(53.2784, -8.934291)","(53.277836, -8.962277)",1867.861418,NaN,5,False,0,1
108,334455,2,2024-01-02 08:09:00,1,120.0,1,NaN,NaN,NaN,"(53.278965, -8.906306)","(53.2784, -8.934291)",1867.773912,NaN,6,False,0,1
109,334455,2,2024-01-02 08:11:00,1,120.0,1,NaN,NaN,NaN,"(53.279529, -8.878321)","(53.278965, -8.906306)",1867.745587,NaN,7,False,0,1


### 2.11 Time preriods between odometer reporting 

In [879]:
timecum = sample.loc[(sample['trip_section']>0)&(sample['is_odometer']!= 1)&(sample['odometer_group']!=0)]\
    .groupby(['id', 'trip_id', 'odometer_group'], as_index=False)['timespent'].sum()

In [880]:
timeFirst = sample.loc[(sample['trip_section']!=0)&(sample['is_odometer']== 1)&(sample['odometer_group']!=0)]\
    .groupby(['id', 'trip_id', 'odometer_group'], as_index=False)['timespent'].max()
timeFirst.rename(columns = {'timespent':'quant_time'}, inplace=True)

In [881]:
timeFirst=timeFirst.merge(timecum, on=['id', 'trip_id', 'odometer_group'], how='outer').fillna(0)
timeFirst.sort_values(['id', 'trip_id', 'odometer_group'], inplace=True)

In [882]:
timeFirst['prev_timecum']=timeFirst.sort_values(['id', 'trip_id', 'odometer_group']).groupby(['id', 'trip_id'])['timespent'].shift(1)

In [883]:
timeFirst['ttr']= timeFirst.quant_time+timeFirst.prev_timecum


In [884]:
timeFirst= timeFirst[['id', 'trip_id', 'odometer_group', 'ttr']]


In [885]:
timeFirst

,id,trip_id,odometer_group,ttr
0,334455,1,1,NaN
1,334455,1,2,480.0
2,334455,1,3,2160.0
3,334455,1,4,60.0
4,334455,1,5,1560.0
...,...,...,...,...
1485,334459,3,197,69.0
1486,334459,3,198,41.0
1487,334459,3,199,65.0
1488,334459,3,200,51.0


In [886]:
od_values=od_values.merge(timecum, on = ['id', 'trip_id', 'odometer_group'], how = 'left')
# sample=sample.merge(min_val, on = ['id', 'trip_id'])


In [887]:
od_values=od_values.merge(timeFirst, on=['id', 'trip_id', 'odometer_group'])
od_values['avg_od_speed']=od_values['od_distance']/od_values['ttr']



In [888]:
od_values['avg_od_speed']=od_values['avg_od_speed'].apply(lambda x: np.nan if x<0 else x)

In [889]:
od_values['avg_od_speed']=od_values['avg_od_speed'].shift(-1)

In [890]:
od_values= od_values[['id', 'trip_id', 'odometer_group', 'avg_od_speed']]
od_values.drop_duplicates(subset=['id', 'trip_id', 'odometer_group'], inplace=True)

In [891]:
#sample.loc[(sample.is_odometer==1)]['avg_od_speed'].shift(1)

In [892]:
sample=sample.merge(od_values, on=['id','trip_id', 'odometer_group'], how = 'left')

In [893]:
s=sample.loc[(sample.is_odometer==1)]['avg_od_speed'].shift(1)
sample=sample.join(s, how='left', rsuffix='_2')

In [894]:
sample['avg_od_speed']= sample.apply(lambda x: x['avg_od_speed'] if x['is_odometer']==0 else x['avg_od_speed_2'] , axis=1)

In [895]:
sample['od_speed']=sample['od_distance']/sample['timespent']
sample['gps_speed']=sample['gps_distance']/sample['timespent']

In [896]:
sample.loc[sample.trip_section==0, ['gps_distance']]=0
sample.loc[sample.trip_section==0, ['od_distance']]=0
sample.loc[sample.trip_section==0, ['avg_od_speed']]=0
sample.loc[sample.trip_section==0, ['od_speed']]=0
sample.loc[sample.trip_section==0, ['gps_speed']]=0

In [897]:
sample.columns

Index(['id', 'trip_id', 'timestamp', 'event_sec_id', 'timespent',
       'ignition_state', 'curr_od_km', 'prev_od_km', 'od_distance',
       'current_loc', 'prev_loc', 'gps_distance', 'fuel_%', 'trip_section',
       'odometer_valuelability', 'is_odometer', 'odometer_group',
       'avg_od_speed', 'avg_od_speed_2', 'od_speed', 'gps_speed'],
      dtype='object')

## Check how valueble reports from odometer

In [898]:
sample.groupby(['id', 'odometer_valuelability'])['id'].count()

id      odometer_valuelability
334455  False                     204
334456  True                      252
334457  False                     278
        True                      202
334458  True                      355
334459  True                      730
Name: id, dtype: int64

In [899]:
#sample.loc[abs(sample.od_distance - sample.gps_distance)>1000 ]
sample.loc[(sample.gps_speed>70) & (sample.od_speed>70)]
# sample.loc[]

,id,trip_id,timestamp,event_sec_id,timespent,ignition_state,curr_od_km,prev_od_km,od_distance,current_loc,...,gps_distance,fuel_%,trip_section,odometer_valuelability,is_odometer,odometer_group,avg_od_speed,avg_od_speed_2,od_speed,gps_speed
219,334456,1,2024-01-04 10:22:03,1,18.0,1,23.68,22.20,1480.0,"(53.912506, -8.279932)",...,1481.436197,85.0,15,True,1,16,82.222222,82.222222,82.222222,82.302011
242,334456,1,2024-01-04 10:55:51,1,4.0,1,57.72,56.24,1480.0,"(53.627785, -8.089387)",...,1482.737011,62.0,38,True,1,39,370.000000,370.000000,370.000000,370.684253
246,334456,1,2024-01-04 11:02:30,1,5.0,1,63.64,62.16,1480.0,"(53.578268, -8.056248)",...,1482.988033,58.0,42,True,1,43,296.000000,296.000000,296.000000,296.597607
254,334456,1,2024-01-04 11:12:49,1,3.0,1,75.48,74.00,1480.0,"(53.479235, -7.989972)",...,1483.441426,50.0,50,True,1,51,493.333333,493.333333,493.333333,494.480475
258,334456,1,2024-01-04 11:19:00,1,5.0,1,81.40,79.92,1480.0,"(53.429718, -7.956834)",...,1483.643586,46.0,54,True,1,55,296.000000,296.000000,296.000000,296.728717
259,334456,1,2024-01-04 11:19:12,1,12.0,1,82.88,81.40,1480.0,"(53.417339, -7.948549)",...,1483.724966,45.0,55,True,1,56,123.333333,123.333333,123.333333,123.643747
260,334456,1,2024-01-04 11:19:20,1,8.0,1,84.36,82.88,1480.0,"(53.40496, -7.940265)",...,1483.756999,44.0,56,True,1,57,185.000000,185.000000,185.000000,185.469625
270,334456,1,2024-01-04 11:35:37,1,10.0,1,99.16,97.68,1480.0,"(53.281168, -7.857419)",...,1484.349201,34.0,66,True,1,67,148.000000,148.000000,148.000000,148.434920
284,334456,1,2024-01-04 11:58:27,1,13.0,1,119.88,118.40,1480.0,"(53.107859, -7.741435)",...,1485.144553,90.0,80,True,1,81,113.846154,113.846154,113.846154,114.241889
289,334456,1,2024-01-04 12:06:25,1,17.0,1,127.28,125.80,1480.0,"(53.045963, -7.700012)",...,1485.428838,85.0,85,True,1,86,87.058824,87.058824,87.058824,87.378167


## 2.12 To choose one distance and speed. 

In [900]:
#let's set condition, that it's rather not possible our vehicles will maintain 250 km/h ()
speed_limit = 250_000/3600 

def distance_choicer(df_row):
    dist = 0 
    if df_row['odometer_valuelability'] and df_row['od_distance']>=0 and not(np.isnan(df_row['od_distance']))\
        and df_row['od_speed'] <= speed_limit:
            dist = df_row['od_distance']
    elif df_row['gps_distance']>=0 and df_row['gps_speed']>=0 and  not(np.isnan(df_row['gps_distance']))\
          and df_row['gps_speed']<= speed_limit:
            dist = df_row['gps_distance']
    return dist

def speed_choicer(df_row):
    spd = 0 
    if df_row['odometer_valuelability'] and df_row['od_distance']>=0 and not(np.isnan(df_row['od_speed']))\
        and df_row['od_speed'] <= speed_limit:
            spd = df_row['od_speed']
    elif df_row['gps_distance']>=0 and df_row['gps_speed']>=0 and  not(np.isnan(df_row['gps_speed']))\
          and df_row['gps_speed']<= speed_limit:
            spd = df_row['gps_speed']
    return spd



sample['distance'] = sample.apply(distance_choicer, axis=1)
sample['speed'] = sample.apply(speed_choicer, axis= 1)
    
    
    

In [901]:
sample

,id,trip_id,timestamp,event_sec_id,timespent,ignition_state,curr_od_km,prev_od_km,od_distance,current_loc,...,trip_section,odometer_valuelability,is_odometer,odometer_group,avg_od_speed,avg_od_speed_2,od_speed,gps_speed,distance,speed
0,334455,1,2024-01-01 15:00:00,1,NaN,1,NaN,NaN,0.0,"(53.332025, -6.275705)",...,0,False,0,0,0.000000,NaN,0.000000,0.000000,0.000000,0.000000
1,334455,1,2024-01-01 15:04:00,1,240.0,1,NaN,NaN,NaN,"(53.331461, -6.30369)",...,1,False,0,0,NaN,NaN,NaN,7.772745,1865.458790,7.772745
2,334455,1,2024-01-01 15:07:00,1,180.0,1,NaN,NaN,NaN,"(53.330896, -6.331675)",...,2,False,0,0,NaN,NaN,NaN,10.363817,1865.487137,10.363817
3,334455,1,2024-01-01 15:09:00,1,120.0,1,NaN,NaN,NaN,"(53.330332, -6.35966)",...,3,False,0,0,NaN,NaN,NaN,15.545900,1865.507988,15.545900
4,334455,1,2024-01-01 15:12:00,1,180.0,1,NaN,NaN,NaN,"(53.329767, -6.387645)",...,4,False,0,0,NaN,NaN,NaN,10.364091,1865.536334,10.364091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016,334459,3,2024-01-07 19:02:53,1,69.0,1,5107.81,5105.95,1860.0,"(50.145249, 8.641091)",...,196,True,1,197,26.956522,26.956522,26.956522,27.475628,1860.000000,26.956522
2017,334459,3,2024-01-07 19:03:34,1,41.0,1,5109.67,5107.81,1860.0,"(50.135518, 8.662869)",...,197,True,1,198,45.365854,45.365854,45.365854,46.247202,1860.000000,45.365854
2018,334459,3,2024-01-07 19:04:39,1,65.0,1,5111.53,5109.67,1860.0,"(50.125787, 8.684646)",...,198,True,1,199,28.615385,28.615385,28.615385,29.174382,1860.000000,28.615385
2019,334459,3,2024-01-07 19:05:30,1,51.0,1,5113.39,5111.53,1860.0,"(50.116055, 8.706423)",...,199,True,1,200,36.470588,36.470588,36.470588,37.189343,1860.000000,36.470588


In [902]:
sample1

,id,trip_id,timestamp,event_sec_id,timespent,ignition_state,curr_od_km,prev_od_km,od_distance,current_loc,...,trip_section,odometer_valuelability,is_odometer,odometer_group,avg_od_speed,avg_od_speed_2,od_speed,gps_speed,distance,speed
0,334455,1,2024-01-01 15:00:00,1,NaN,1,NaN,NaN,0.0,"(53.332025, -6.275705)",...,0,False,0,0,0.000000,NaN,0.000000,0.000000,0.000000,0.000000
1,334455,1,2024-01-01 15:04:00,1,240.0,1,NaN,NaN,NaN,"(53.331461, -6.30369)",...,1,False,0,0,NaN,NaN,NaN,7.772745,1865.458790,7.772745
2,334455,1,2024-01-01 15:07:00,1,180.0,1,NaN,NaN,NaN,"(53.330896, -6.331675)",...,2,False,0,0,NaN,NaN,NaN,10.363817,1865.487137,10.363817
3,334455,1,2024-01-01 15:09:00,1,120.0,1,NaN,NaN,NaN,"(53.330332, -6.35966)",...,3,False,0,0,NaN,NaN,NaN,15.545900,1865.507988,15.545900
4,334455,1,2024-01-01 15:12:00,1,180.0,1,NaN,NaN,NaN,"(53.329767, -6.387645)",...,4,False,0,0,NaN,NaN,NaN,10.364091,1865.536334,10.364091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016,334459,3,2024-01-07 19:02:53,1,69.0,1,5107.81,5105.95,1860.0,"(50.145249, 8.641091)",...,196,True,1,197,26.956522,26.956522,26.956522,27.475628,1860.000000,26.956522
2017,334459,3,2024-01-07 19:03:34,1,41.0,1,5109.67,5107.81,1860.0,"(50.135518, 8.662869)",...,197,True,1,198,45.365854,45.365854,45.365854,46.247202,1860.000000,45.365854
2018,334459,3,2024-01-07 19:04:39,1,65.0,1,5111.53,5109.67,1860.0,"(50.125787, 8.684646)",...,198,True,1,199,28.615385,28.615385,28.615385,29.174382,1860.000000,28.615385
2019,334459,3,2024-01-07 19:05:30,1,51.0,1,5113.39,5111.53,1860.0,"(50.116055, 8.706423)",...,199,True,1,200,36.470588,36.470588,36.470588,37.189343,1860.000000,36.470588


In [903]:
sample

,id,trip_id,timestamp,event_sec_id,timespent,ignition_state,curr_od_km,prev_od_km,od_distance,current_loc,...,trip_section,odometer_valuelability,is_odometer,odometer_group,avg_od_speed,avg_od_speed_2,od_speed,gps_speed,distance,speed
0,334455,1,2024-01-01 15:00:00,1,NaN,1,NaN,NaN,0.0,"(53.332025, -6.275705)",...,0,False,0,0,0.000000,NaN,0.000000,0.000000,0.000000,0.000000
1,334455,1,2024-01-01 15:04:00,1,240.0,1,NaN,NaN,NaN,"(53.331461, -6.30369)",...,1,False,0,0,NaN,NaN,NaN,7.772745,1865.458790,7.772745
2,334455,1,2024-01-01 15:07:00,1,180.0,1,NaN,NaN,NaN,"(53.330896, -6.331675)",...,2,False,0,0,NaN,NaN,NaN,10.363817,1865.487137,10.363817
3,334455,1,2024-01-01 15:09:00,1,120.0,1,NaN,NaN,NaN,"(53.330332, -6.35966)",...,3,False,0,0,NaN,NaN,NaN,15.545900,1865.507988,15.545900
4,334455,1,2024-01-01 15:12:00,1,180.0,1,NaN,NaN,NaN,"(53.329767, -6.387645)",...,4,False,0,0,NaN,NaN,NaN,10.364091,1865.536334,10.364091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016,334459,3,2024-01-07 19:02:53,1,69.0,1,5107.81,5105.95,1860.0,"(50.145249, 8.641091)",...,196,True,1,197,26.956522,26.956522,26.956522,27.475628,1860.000000,26.956522
2017,334459,3,2024-01-07 19:03:34,1,41.0,1,5109.67,5107.81,1860.0,"(50.135518, 8.662869)",...,197,True,1,198,45.365854,45.365854,45.365854,46.247202,1860.000000,45.365854
2018,334459,3,2024-01-07 19:04:39,1,65.0,1,5111.53,5109.67,1860.0,"(50.125787, 8.684646)",...,198,True,1,199,28.615385,28.615385,28.615385,29.174382,1860.000000,28.615385
2019,334459,3,2024-01-07 19:05:30,1,51.0,1,5113.39,5111.53,1860.0,"(50.116055, 8.706423)",...,199,True,1,200,36.470588,36.470588,36.470588,37.189343,1860.000000,36.470588


In [904]:
sample.columns

Index(['id', 'trip_id', 'timestamp', 'event_sec_id', 'timespent',
       'ignition_state', 'curr_od_km', 'prev_od_km', 'od_distance',
       'current_loc', 'prev_loc', 'gps_distance', 'fuel_%', 'trip_section',
       'odometer_valuelability', 'is_odometer', 'odometer_group',
       'avg_od_speed', 'avg_od_speed_2', 'od_speed', 'gps_speed', 'distance',
       'speed'],
      dtype='object')

In [905]:
sample1.columns

Index(['id', 'trip_id', 'timestamp', 'event_sec_id', 'timespent',
       'ignition_state', 'curr_od_km', 'prev_od_km', 'od_distance',
       'current_loc', 'prev_loc', 'gps_distance', 'fuel_%', 'trip_section',
       'odometer_valuelability', 'is_odometer', 'odometer_group',
       'avg_od_speed', 'avg_od_speed_2', 'od_speed', 'gps_speed', 'distance',
       'speed'],
      dtype='object')

In [906]:
sample.loc[sample.ignition_state==0, 'fuel_%']

101     80.0
203      NaN
657     10.0
784      NaN
1037    80.0
1139     NaN
1517    24.0
1819    30.0
Name: fuel_%, dtype: float64

In [907]:
#sample.loc[np.isnan(sample.speed),'speed']

In [908]:
#clear too big speeds
sample.drop(sample.loc[(sample.gps_speed>speed_limit) & (sample.od_speed>speed_limit)].index, inplace=True)
sample.drop(sample.loc[sample.event_sec_id > 1].index, inplace=True)

In [909]:
sample = sample[['id', 'trip_id', 'trip_section', 'timestamp', 'timespent', 
                 'current_loc', 'prev_loc', 'fuel_%', 'speed', 'distance', 'avg_od_speed', 'gps_speed']]

In [910]:
sample.timespent.fillna(0, inplace=True)
sample.timespent = sample.timespent.apply(int)
sample.speed = sample.speed.apply(int)
sample.distance = sample.distance.apply(int)

C:\Users\Yevhen\AppData\Local\Temp\ipykernel_17496\3182006104.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample.timespent.fillna(0, inplace=True)
C:\Users\Yevhen\AppData\Local\Temp\ipykernel_17496\3182006104.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample.timespent = sample.timespent.apply(int)
C:\Users\Yevhen\AppData\Local\Temp\ipykernel_17496\3182006104.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

In [943]:
def fuel_cleaning(df_input):
    """To clear fuel change in table"""
    
    def fuel_percentage(df_row):
        "To put avg fuel change in table"
        res= np.nan
        if df_row['is_fuel'] == 1:
            res=df_row['fuel_%']
        elif df_row['fuel_%_per_timespent'] >=0: 
            res=(-1)*df_row['timespent']*df_row['fuel_%_per_timespent']
        return res

    df=df_input.copy()
    df['is_fuel'] =df.apply(lambda x:0 if np.isnan(x['fuel_%']) else 1, axis= 1)
    df['fuel_seq']=df.sort_values(['id', 'trip_id', 'trip_section'])\
                            .groupby(['id', 'trip_id'])['is_fuel'].cumsum()
    df['fuel_%'] = df.sort_values(['id', 'trip_id', 'timestamp']).groupby(['id', 'trip_id'])['fuel_%']\
            .fillna(method='ffill')
    df_groupby = df.groupby(['id', 'trip_id', 'fuel_seq', 'fuel_%'], as_index=False).agg({ 'trip_section': 'nunique', 
                                                                                    'distance': 'sum', 
                                                                                    'timespent': 'sum'
                                                                             })
    df_groupby['delta_fuel'] = df_groupby.sort_values(['id', 'trip_id', 'fuel_seq'])\
                                .groupby(['id', 'trip_id'])['fuel_%'].transform(lambda x: x - x.shift(-1))
    
    df_groupby['fuel_%_per_timespent'] = df_groupby.delta_fuel/df_groupby.timespent
    df_groupby['fuel_%_per_distance'] = df_groupby.delta_fuel/df_groupby.distance
    df_groupby= df_groupby[['id', 'trip_id', 'fuel_seq', 'delta_fuel', 'fuel_%_per_distance', 'fuel_%_per_timespent']]
    df=df.merge(df_groupby, on=['id', 'trip_id', 'fuel_seq'], how='left')
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    
    #Calculate fuel percents for many rows where there is none
    df['fuel_perc_change']= df.apply(fuel_percentage, axis=1)
    
    #Just to fill in the table (forward+some calculation)

    df['fuel_perc_change']=df.sort_values(['id', 'trip_id', 'timestamp', 'fuel_seq'])\
                                .groupby(['id', 'trip_id', 'fuel_seq'])['fuel_perc_change'].cumsum()
    df['fuel_spent%_per_time_spent']= df['fuel_%_per_timespent'].apply(lambda x: x if x>=0 else np.nan)
    df['fuel_spent%_per_time_spent']= df.groupby('id')['fuel_spent%_per_time_spent'].fillna(method='ffill')
    df['spent_fuel%'] = df['fuel_spent%_per_time_spent']*df['timespent']
    df['spent_fuel%']=df['spent_fuel%'].shift(1)
    df['spent_fuel%']=df['spent_fuel%'].fillna(0)
    
    return df
    
    


In [952]:
test = fuel_cleaning(sample1)

proved_fuel_spent = test.loc[sample_fuel.delta_fuel >= 0].groupby(['id', 'trip_id', 'delta_fuel', 'fuel_seq'], as_index=False)['fuel_perc_change'].count()
proved_fuel_spent=test.groupby(['id', 'trip_id'], as_index=False)['delta_fuel'].sum()

proved_fuel_refill =  test.loc[sample_fuel.delta_fuel < 0].groupby(['id', 'trip_id', 'delta_fuel', 'fuel_seq'], as_index=False)['fuel_perc_change'].count()
proved_fuel_refill= test.groupby(['id', 'trip_id'], as_index=False)['delta_fuel'].sum()

In [945]:
test

,id,trip_id,trip_section,timestamp,timespent,current_loc,prev_loc,fuel_%,speed,distance,avg_od_speed,gps_speed,is_fuel,fuel_seq,delta_fuel,fuel_%_per_distance,fuel_%_per_timespent,fuel_perc_change,fuel_spent%_per_time_spent,spent_fuel%
0,334455,1,0,2024-01-01 15:00:00,NaN,"(53.332025, -6.275705)",NaN,100.0,0.000000,0.000000,0.000000,0.000000,1,1,20.0,0.000412,0.004274,100.000000,0.004274,0.000000
1,334455,1,1,2024-01-01 15:04:00,240.0,"(53.331461, -6.30369)","(53.332025, -6.275705)",100.0,7.772745,1865.458790,NaN,7.772745,0,1,20.0,0.000412,0.004274,98.974359,0.004274,0.000000
2,334455,1,2,2024-01-01 15:07:00,180.0,"(53.330896, -6.331675)","(53.331461, -6.30369)",100.0,10.363817,1865.487137,NaN,10.363817,0,1,20.0,0.000412,0.004274,98.205128,0.004274,1.025641
3,334455,1,3,2024-01-01 15:09:00,120.0,"(53.330332, -6.35966)","(53.330896, -6.331675)",100.0,15.545900,1865.507988,NaN,15.545900,0,1,20.0,0.000412,0.004274,97.692308,0.004274,0.769231
4,334455,1,4,2024-01-01 15:12:00,180.0,"(53.329767, -6.387645)","(53.330332, -6.35966)",100.0,10.364091,1865.536334,NaN,10.364091,0,1,20.0,0.000412,0.004274,96.923077,0.004274,0.512821
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1969,334459,3,196,2024-01-07 19:02:53,69.0,"(50.145249, 8.641091)","(50.15498, 8.619314)",51.0,26.956522,1860.000000,26.956522,27.475628,1,190,0.0,0.000000,0.000000,51.000000,0.000000,0.000000
1970,334459,3,197,2024-01-07 19:03:34,41.0,"(50.135518, 8.662869)","(50.145249, 8.641091)",51.0,45.365854,1860.000000,45.365854,46.247202,1,191,0.0,0.000000,0.000000,51.000000,0.000000,0.000000
1971,334459,3,198,2024-01-07 19:04:39,65.0,"(50.125787, 8.684646)","(50.135518, 8.662869)",51.0,28.615385,1860.000000,28.615385,29.174382,1,192,1.0,0.000538,0.015385,51.000000,0.015385,0.000000
1972,334459,3,199,2024-01-07 19:05:30,51.0,"(50.116055, 8.706423)","(50.125787, 8.684646)",50.0,36.470588,1860.000000,36.470588,37.189343,1,193,0.0,0.000000,0.000000,50.000000,0.000000,1.000000


In [911]:
sample_fuel = sample.copy()

In [912]:
sample_fuel['is_fuel'] =sample_fuel.apply(lambda x:0 if np.isnan(x['fuel_%']) else 1, axis= 1)

sample_fuel['fuel_seq']=sample_fuel.sort_values(['id', 'trip_id', 'trip_section'])\
                        .groupby(['id', 'trip_id'])['is_fuel'].cumsum()

In [913]:
sample_fuel['fuel_%'] = sample_fuel.sort_values(['id', 'trip_id', 'timestamp']).groupby(['id', 'trip_id'])['fuel_%']\
            .fillna(method='ffill')

In [914]:
sample_fuel

,id,trip_id,trip_section,timestamp,timespent,current_loc,prev_loc,fuel_%,speed,distance,avg_od_speed,gps_speed,is_fuel,fuel_seq
0,334455,1,0,2024-01-01 15:00:00,0,"(53.332025, -6.275705)",NaN,100.0,0,0,0.000000,0.000000,1,1
1,334455,1,1,2024-01-01 15:04:00,240,"(53.331461, -6.30369)","(53.332025, -6.275705)",100.0,7,1865,NaN,7.772745,0,1
2,334455,1,2,2024-01-01 15:07:00,180,"(53.330896, -6.331675)","(53.331461, -6.30369)",100.0,10,1865,NaN,10.363817,0,1
3,334455,1,3,2024-01-01 15:09:00,120,"(53.330332, -6.35966)","(53.330896, -6.331675)",100.0,15,1865,NaN,15.545900,0,1
4,334455,1,4,2024-01-01 15:12:00,180,"(53.329767, -6.387645)","(53.330332, -6.35966)",100.0,10,1865,NaN,10.364091,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016,334459,3,196,2024-01-07 19:02:53,69,"(50.145249, 8.641091)","(50.15498, 8.619314)",51.0,26,1859,26.956522,27.475628,1,190
2017,334459,3,197,2024-01-07 19:03:34,41,"(50.135518, 8.662869)","(50.145249, 8.641091)",51.0,45,1859,45.365854,46.247202,1,191
2018,334459,3,198,2024-01-07 19:04:39,65,"(50.125787, 8.684646)","(50.135518, 8.662869)",51.0,28,1859,28.615385,29.174382,1,192
2019,334459,3,199,2024-01-07 19:05:30,51,"(50.116055, 8.706423)","(50.125787, 8.684646)",50.0,36,1859,36.470588,37.189343,1,193


In [915]:
sample_groupby = sample_fuel.groupby(['id', 'trip_id', 'fuel_seq', 'fuel_%'], as_index=False).agg({ 'trip_section': 'nunique', 
                                                                                    'distance': 'sum', 
                                                                                    'timespent': 'sum'
                                                                             })

#agg({'B': ['min', 'max'], 'C': 'sum'})

In [917]:
sample_groupby['delta_fuel'] = sample_groupby.sort_values(['id', 'trip_id', 'fuel_seq'])\
                                .groupby(['id', 'trip_id'])['fuel_%'].transform(lambda x: x - x.shift(-1))
sample_groupby

,id,trip_id,fuel_seq,fuel_%,trip_section,distance,timespent,delta_fuel
0,334455,1,1,100.0,27,48495,4680,20.0
1,334455,1,2,80.0,32,59712,5820,20.0
2,334455,1,3,60.0,26,48538,4140,20.0
3,334455,1,4,40.0,2,3734,240,-60.0
4,334455,1,5,100.0,13,24271,2520,20.0
...,...,...,...,...,...,...,...,...
987,334459,3,190,51.0,1,1859,69,0.0
988,334459,3,191,51.0,1,1859,41,0.0
989,334459,3,192,51.0,1,1859,65,1.0
990,334459,3,193,50.0,1,1859,51,0.0


In [918]:
sample_groupby['fuel_%_per_timespent'] = sample_groupby.delta_fuel/sample_groupby.timespent
sample_groupby['fuel_%_per_distance'] = sample_groupby.delta_fuel/sample_groupby.distance

In [919]:
sample_groupby= sample_groupby[['id', 'trip_id', 'fuel_seq', 'delta_fuel', 'fuel_%_per_distance', 'fuel_%_per_timespent']]

In [920]:
sample_groupby

,id,trip_id,fuel_seq,delta_fuel,fuel_%_per_distance,fuel_%_per_timespent
0,334455,1,1,20.0,0.000412,0.004274
1,334455,1,2,20.0,0.000335,0.003436
2,334455,1,3,20.0,0.000412,0.004831
3,334455,1,4,-60.0,-0.016069,-0.250000
4,334455,1,5,20.0,0.000824,0.007937
...,...,...,...,...,...,...
987,334459,3,190,0.0,0.000000,0.000000
988,334459,3,191,0.0,0.000000,0.000000
989,334459,3,192,1.0,0.000538,0.015385
990,334459,3,193,0.0,0.000000,0.000000


In [921]:
sample_fuel=sample_fuel.merge(sample_groupby, on=['id', 'trip_id', 'fuel_seq'], how='left')

In [922]:
sample_fuel.replace([np.inf, -np.inf], np.nan, inplace=True)

In [923]:
def fuel_percentage(df_row):
    res= np.nan
    if df_row['is_fuel'] == 1:
        res=df_row['fuel_%']
    elif df_row['fuel_%_per_timespent'] >=0: 
        res=(-1)*df_row['timespent']*df_row['fuel_%_per_timespent']
    return res

In [924]:
sample_fuel['fuel_perc_change']= sample_fuel.apply(fuel_percentage, axis=1)


#(lambda x: x['fuel_%'] if x['is_fuel']==1 else x['timespent']*(-1)*x['fuel_%_per_timespent'], axis=1)
sample_fuel['fuel_perc_change']=sample_fuel.sort_values(['id', 'trip_id', 'timestamp', 'fuel_seq'])\
                            .groupby(['id', 'trip_id', 'fuel_seq'])['fuel_perc_change'].cumsum()


# sample_fuel['fuel']

In [925]:
sample_fuel['fuel_spent%_per_time_spent']= sample_fuel['fuel_%_per_timespent'].apply(lambda x: x if x>=0 else np.nan)
sample_fuel['fuel_spent%_per_time_spent']= sample_fuel.groupby('id')['fuel_spent%_per_time_spent'].fillna(method='ffill')
sample_fuel['spent_fuel%'] = sample_fuel['fuel_spent%_per_time_spent']*sample_fuel['timespent']

In [926]:
sample_fuel['spent_fuel%']=sample_fuel['spent_fuel%'].shift(1)
sample_fuel['spent_fuel%']=sample_fuel['spent_fuel%'].fillna(0)

In [937]:
test

,id,trip_id,timestamp,event_sec_id,timespent,ignition_state,curr_od_km,prev_od_km,od_distance,current_loc,...,distance,speed,is_fuel,fuel_seq,delta_fuel,fuel_%_per_distance,fuel_%_per_timespent,fuel_perc_change,fuel_spent%_per_time_spent,spent_fuel%
0,334455,1,2024-01-01 15:00:00,1,NaN,1,NaN,NaN,0.0,"(53.332025, -6.275705)",...,0.000000,0.000000,1,1,20.0,0.000412,0.004274,100.000000,0.004274,0.000000
1,334455,1,2024-01-01 15:04:00,1,240.0,1,NaN,NaN,NaN,"(53.331461, -6.30369)",...,1865.458790,7.772745,0,1,20.0,0.000412,0.004274,98.974359,0.004274,0.000000
2,334455,1,2024-01-01 15:07:00,1,180.0,1,NaN,NaN,NaN,"(53.330896, -6.331675)",...,1865.487137,10.363817,0,1,20.0,0.000412,0.004274,98.205128,0.004274,1.025641
3,334455,1,2024-01-01 15:09:00,1,120.0,1,NaN,NaN,NaN,"(53.330332, -6.35966)",...,1865.507988,15.545900,0,1,20.0,0.000412,0.004274,97.692308,0.004274,0.769231
4,334455,1,2024-01-01 15:12:00,1,180.0,1,NaN,NaN,NaN,"(53.329767, -6.387645)",...,1865.536334,10.364091,0,1,20.0,0.000412,0.004274,96.923077,0.004274,0.512821
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1969,334459,3,2024-01-07 19:02:53,1,69.0,1,5107.81,5105.95,1860.0,"(50.145249, 8.641091)",...,1860.000000,26.956522,1,190,0.0,0.000000,0.000000,51.000000,0.000000,0.000000
1970,334459,3,2024-01-07 19:03:34,1,41.0,1,5109.67,5107.81,1860.0,"(50.135518, 8.662869)",...,1860.000000,45.365854,1,191,0.0,0.000000,0.000000,51.000000,0.000000,0.000000
1971,334459,3,2024-01-07 19:04:39,1,65.0,1,5111.53,5109.67,1860.0,"(50.125787, 8.684646)",...,1860.000000,28.615385,1,192,1.0,0.000538,0.015385,51.000000,0.015385,0.000000
1972,334459,3,2024-01-07 19:05:30,1,51.0,1,5113.39,5111.53,1860.0,"(50.116055, 8.706423)",...,1860.000000,36.470588,1,193,0.0,0.000000,0.000000,50.000000,0.000000,1.000000


In [930]:
sample_fuel.groupby(['id', 'trip_id'])[['spent_fuel%', 'timespent']].sum()

spent_fuel%  timespent
id     trip_id                        
334455 1          81.159420      17580
       2         417.619048      52500
334456 1         249.944055      21564
334457 1         148.004422      16131
       2         159.278751      95444
334458 1          81.801622      17580
       2         102.485175      58536
334459 1          56.301887      12528
       2          75.000000      95809
       3          50.487179      89841

In [ ]:
proved_fuel_refill =  test.loc[sample_fuel.delta_fuel < 0].groupby(['id', 'trip_id', 'delta_fuel', 'fuel_seq'], as_index=False)['fuel_perc_change'].count()
proved_fuel_refill= test.groupby(['id', 'trip_id'], as_index=False)['delta_fuel'].sum()

In [951]:
test == sample_fuel

,id,trip_id,trip_section,timestamp,timespent,current_loc,prev_loc,fuel_%,speed,distance,avg_od_speed,gps_speed,is_fuel,fuel_seq,delta_fuel,fuel_%_per_distance,fuel_%_per_timespent,fuel_perc_change,fuel_spent%_per_time_spent,spent_fuel%
0,True,True,True,True,True,True,False,True,True,True,True,True,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True,True,True,False,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,True,False,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True,True,False,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True,True,False,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1969,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
1970,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
1971,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
1972,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True


In [949]:
test['delta_fuel'].unique()

array([  20.,  -60.,   nan,    1.,    2.,    4.,  -68.,   -1.,    0.,
        -76.,  -40.,   10., -100.,   16.,  -88.,   22.,   18.,  -81.])

In [932]:
proved_fuel_change

,id,trip_id,delta_fuel
0,334455,1,80.0
1,334456,1,244.0
2,334457,1,130.0
3,334457,2,158.0
4,334458,1,80.0
5,334458,2,60.0
6,334459,1,56.0
7,334459,2,75.0
8,334459,3,50.0


In [948]:
# sample_fuel==


test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1974 entries, 0 to 1973
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   id                          1974 non-null   int64         
 1   trip_id                     1974 non-null   int64         
 2   trip_section                1974 non-null   int64         
 3   timestamp                   1974 non-null   datetime64[ns]
 4   timespent                   1969 non-null   float64       
 5   current_loc                 1974 non-null   object        
 6   prev_loc                    1969 non-null   object        
 7   fuel_%                      1873 non-null   float64       
 8   speed                       1974 non-null   float64       
 9   distance                    1974 non-null   float64       
 10  avg_od_speed                1948 non-null   float64       
 11  gps_speed                   1968 non-null   float64     

In [939]:
test

,id,trip_id,timestamp,event_sec_id,timespent,ignition_state,curr_od_km,prev_od_km,od_distance,current_loc,...,distance,speed,is_fuel,fuel_seq,delta_fuel,fuel_%_per_distance,fuel_%_per_timespent,fuel_perc_change,fuel_spent%_per_time_spent,spent_fuel%
0,334455,1,2024-01-01 15:00:00,1,NaN,1,NaN,NaN,0.0,"(53.332025, -6.275705)",...,0.000000,0.000000,1,1,20.0,0.000412,0.004274,100.000000,0.004274,0.000000
1,334455,1,2024-01-01 15:04:00,1,240.0,1,NaN,NaN,NaN,"(53.331461, -6.30369)",...,1865.458790,7.772745,0,1,20.0,0.000412,0.004274,98.974359,0.004274,0.000000
2,334455,1,2024-01-01 15:07:00,1,180.0,1,NaN,NaN,NaN,"(53.330896, -6.331675)",...,1865.487137,10.363817,0,1,20.0,0.000412,0.004274,98.205128,0.004274,1.025641
3,334455,1,2024-01-01 15:09:00,1,120.0,1,NaN,NaN,NaN,"(53.330332, -6.35966)",...,1865.507988,15.545900,0,1,20.0,0.000412,0.004274,97.692308,0.004274,0.769231
4,334455,1,2024-01-01 15:12:00,1,180.0,1,NaN,NaN,NaN,"(53.329767, -6.387645)",...,1865.536334,10.364091,0,1,20.0,0.000412,0.004274,96.923077,0.004274,0.512821
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1969,334459,3,2024-01-07 19:02:53,1,69.0,1,5107.81,5105.95,1860.0,"(50.145249, 8.641091)",...,1860.000000,26.956522,1,190,0.0,0.000000,0.000000,51.000000,0.000000,0.000000
1970,334459,3,2024-01-07 19:03:34,1,41.0,1,5109.67,5107.81,1860.0,"(50.135518, 8.662869)",...,1860.000000,45.365854,1,191,0.0,0.000000,0.000000,51.000000,0.000000,0.000000
1971,334459,3,2024-01-07 19:04:39,1,65.0,1,5111.53,5109.67,1860.0,"(50.125787, 8.684646)",...,1860.000000,28.615385,1,192,1.0,0.000538,0.015385,51.000000,0.015385,0.000000
1972,334459,3,2024-01-07 19:05:30,1,51.0,1,5113.39,5111.53,1860.0,"(50.116055, 8.706423)",...,1860.000000,36.470588,1,193,0.0,0.000000,0.000000,50.000000,0.000000,1.000000


In [721]:
sample_fuel.loc[sample_fuel.fuel_seq==2]

,id,trip_id,trip_section,timestamp,timespent,current_loc,prev_loc,fuel_%,speed,distance,is_fuel,fuel_seq,delta_fuel,fuel_%_per_distance,fuel_%_per_timespent,fuel_perc_change,fuel_spent%_per_time_spent,spent_fuel%
27,334455,1,27,2024-01-01 16:21:00,180,"(53.316784, -7.031303)","(53.317349, -7.003318)",80.0,10,1866,1,2,20.0,0.000335,0.003436,80.000000,0.003436,0.256410
28,334455,1,28,2024-01-01 16:22:00,60,"(53.31622, -7.059288)","(53.316784, -7.031303)",80.0,31,1866,0,2,20.0,0.000335,0.003436,79.793814,0.003436,0.618557
29,334455,1,29,2024-01-01 16:24:00,120,"(53.315655, -7.087273)","(53.31622, -7.059288)",80.0,15,1866,0,2,20.0,0.000335,0.003436,79.381443,0.003436,0.206186
30,334455,1,30,2024-01-01 16:25:00,60,"(53.315091, -7.115259)","(53.315655, -7.087273)",80.0,31,1866,0,2,20.0,0.000335,0.003436,79.175258,0.003436,0.412371
31,334455,1,31,2024-01-01 16:30:00,300,"(53.314526, -7.143244)","(53.315091, -7.115259)",80.0,6,1866,0,2,20.0,0.000335,0.003436,78.144330,0.003436,0.206186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1059,334458,2,58,2024-01-02 09:25:00,60,"(53.681138, -1.490539)","(53.66031, -1.496775)",60.0,39,2354,0,2,20.0,0.000230,0.005848,41.052632,0.005848,0.350877
1060,334458,2,59,2024-01-02 09:26:00,60,"(53.701966, -1.484303)","(53.681138, -1.490539)",60.0,39,2354,0,2,20.0,0.000230,0.005848,40.701754,0.005848,0.350877
1254,334459,1,1,2024-01-05 12:01:07,67,"(52.043929, 4.392128)","(52.052579, 4.372771)",80.0,24,1650,1,2,0.0,0.000000,0.000000,80.000000,0.000000,0.000000
1480,334459,2,1,2024-01-06 13:00:41,41,"(50.112812, 8.713682)","(50.106324, 8.7282)",24.0,30,1239,1,2,0.0,0.000000,0.000000,24.000000,0.000000,0.000000


In [690]:
sample_fuel.to_csv('test_fuel.csv')

In [651]:
sample_fuel

,id,trip_id,trip_section,timestamp,timespent,current_loc,prev_loc,fuel_%,speed,distance,is_fuel,fuel_seq,delta_fuel,fuel_%_per_distance,fuel_%_per_timespent
0,334455,1,0,2024-01-01 15:00:00,0,"(53.332025, -6.275705)",NaN,100.0,0,0,1,1,20.0,0.000412,0.004274
1,334455,1,1,2024-01-01 15:04:00,240,"(53.331461, -6.30369)","(53.332025, -6.275705)",100.0,7,1865,0,1,20.0,0.000412,0.004274
2,334455,1,2,2024-01-01 15:07:00,180,"(53.330896, -6.331675)","(53.331461, -6.30369)",100.0,10,1865,0,1,20.0,0.000412,0.004274
3,334455,1,3,2024-01-01 15:09:00,120,"(53.330332, -6.35966)","(53.330896, -6.331675)",100.0,15,1865,0,1,20.0,0.000412,0.004274
4,334455,1,4,2024-01-01 15:12:00,180,"(53.329767, -6.387645)","(53.330332, -6.35966)",100.0,10,1865,0,1,20.0,0.000412,0.004274
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1969,334459,3,196,2024-01-07 19:02:53,69,"(50.145249, 8.641091)","(50.15498, 8.619314)",51.0,26,1859,1,190,0.0,0.000000,0.000000
1970,334459,3,197,2024-01-07 19:03:34,41,"(50.135518, 8.662869)","(50.145249, 8.641091)",51.0,45,1859,1,191,0.0,0.000000,0.000000
1971,334459,3,198,2024-01-07 19:04:39,65,"(50.125787, 8.684646)","(50.135518, 8.662869)",51.0,28,1859,1,192,1.0,0.000538,0.015385
1972,334459,3,199,2024-01-07 19:05:30,51,"(50.116055, 8.706423)","(50.125787, 8.684646)",50.0,36,1859,1,193,0.0,0.000000,0.000000


In [613]:
sample_fuel.info() 

<class 'pandas.core.frame.DataFrame'>
Index: 1974 entries, 0 to 2020
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id            1974 non-null   int64         
 1   trip_id       1974 non-null   int64         
 2   trip_section  1974 non-null   int64         
 3   timestamp     1974 non-null   datetime64[ns]
 4   timespent     1974 non-null   int64         
 5   current_loc   1974 non-null   object        
 6   prev_loc      1969 non-null   object        
 7   fuel_%        1873 non-null   float64       
 8   speed         1974 non-null   int64         
 9   distance      1974 non-null   int64         
 10  is_fuel       1974 non-null   int64         
 11  fuel_seq      1974 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(8), object(2)
memory usage: 200.5+ KB


In [605]:
groupby_fuel=sample_fuel.groupby([''])

<class 'pandas.core.frame.DataFrame'>
Index: 1974 entries, 0 to 2020
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id            1974 non-null   int64         
 1   trip_id       1974 non-null   int64         
 2   trip_section  1974 non-null   int64         
 3   timestamp     1974 non-null   datetime64[ns]
 4   timespent     1974 non-null   int64         
 5   current_loc   1974 non-null   object        
 6   prev_loc      1969 non-null   object        
 7   fuel_%        1873 non-null   float64       
 8   speed         1974 non-null   int64         
 9   distance      1974 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(6), object(2)
memory usage: 169.6+ KB


In [532]:
sample.columns

Index(['id', 'trip_id', 'timestamp', 'event_sec_id', 'timespent',
       'ignition_state', 'curr_od_km', 'prev_od_km', 'od_distance',
       'current_loc', 'prev_loc', 'gps_distance', 'fuel_%', 'trip_section',
       'odometer_valuelability', 'is_odometer', 'odometer_group',
       'avg_od_speed', 'avg_od_speed_2', 'avg_gps_speed', 'od_speed',
       'gps_speed', 'distance', 'speed'],
      dtype='object')

In [480]:
sample.groupby(['id', 'trip_id', 'trip_section'])['current_loc'].count()

id      trip_id  trip_section
334455  1        0               1
                 1               1
                 2               1
                 3               1
                 4               1
                                ..
334459  3        196             1
                 197             1
                 198             1
                 199             1
                 200             1
Name: current_loc, Length: 1974, dtype: int64

In [481]:
sample.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1974 entries, 0 to 2020
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id            1974 non-null   int64         
 1   trip_id       1974 non-null   int64         
 2   trip_section  1974 non-null   int64         
 3   timestamp     1974 non-null   datetime64[ns]
 4   timespent     1969 non-null   float64       
 5   current_loc   1974 non-null   object        
 6   prev_loc      1969 non-null   object        
 7   fuel_%        992 non-null    float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(2)
memory usage: 138.8+ KB


,id,trip_id,timestamp,event_sec_id,timespent,ignition_state,curr_od_km,prev_od_km,od_distance,current_loc,prev_loc,gps_distance,fuel_%,odometer_valuelability,is_odometer,odometer_group,trip_section,avg_od_speed,avg_od_speed_2
0,334455,1,2024-01-01 15:00:00,1,NaN,1,NaN,NaN,NaN,"(53.332025, -6.275705)",NaN,NaN,100.0,False,0,0,0,NaN,NaN
1,334455,1,2024-01-01 15:04:00,1,240.0,1,NaN,NaN,NaN,"(53.331461, -6.30369)","(53.332025, -6.275705)",1865.458790,NaN,False,0,0,1,NaN,NaN
2,334455,1,2024-01-01 15:07:00,1,180.0,1,NaN,NaN,NaN,"(53.330896, -6.331675)","(53.331461, -6.30369)",1865.487137,NaN,False,0,0,2,NaN,NaN
3,334455,1,2024-01-01 15:09:00,1,120.0,1,NaN,NaN,NaN,"(53.330332, -6.35966)","(53.330896, -6.331675)",1865.507988,NaN,False,0,0,3,NaN,NaN
4,334455,1,2024-01-01 15:12:00,1,180.0,1,NaN,NaN,NaN,"(53.329767, -6.387645)","(53.330332, -6.35966)",1865.536334,NaN,False,0,0,4,NaN,NaN
5,334455,1,2024-01-01 15:16:00,1,240.0,1,NaN,NaN,NaN,"(53.329203, -6.415631)","(53.329767, -6.387645)",1865.623771,NaN,False,0,0,5,NaN,NaN
6,334455,1,2024-01-01 15:17:00,1,60.0,1,NaN,NaN,NaN,"(53.328638, -6.443616)","(53.329203, -6.415631)",1865.585530,NaN,False,0,0,6,NaN,NaN
7,334455,1,2024-01-01 15:19:00,1,120.0,1,15.0,NaN,NaN,"(53.328074, -6.471601)","(53.328638, -6.443616)",1865.606380,NaN,False,1,1,7,NaN,NaN
8,334455,1,2024-01-01 15:24:00,1,300.0,1,NaN,15.0,NaN,"(53.327509, -6.499586)","(53.328074, -6.471601)",1865.634726,NaN,False,0,1,8,8.333333,NaN
9,334455,1,2024-01-01 15:27:00,1,180.0,1,19.0,NaN,NaN,"(53.326945, -6.527571)","(53.327509, -6.499586)",1865.655575,NaN,False,1,2,9,8.333333,8.333333


In [298]:
sample.head(40)

,id,trip_id,timestamp,event_sec_id,timespent,ignition_state,curr_od_km,prev_od_km,od_distance,current_loc,prev_loc,gps_distance,fuel_%,odometer_valuelability,is_odometer,odometer_group,trip_section,avg_od_speed
0,334455,1,2024-01-01 15:00:00,1,NaN,1,NaN,NaN,NaN,"(53.332025, -6.275705)",NaN,NaN,100.0,False,0,0,0,NaN
1,334455,1,2024-01-01 15:04:00,1,240.0,1,NaN,NaN,NaN,"(53.331461, -6.30369)","(53.332025, -6.275705)",1865.458790,NaN,False,0,0,1,NaN
2,334455,1,2024-01-01 15:07:00,1,180.0,1,NaN,NaN,NaN,"(53.330896, -6.331675)","(53.331461, -6.30369)",1865.487137,NaN,False,0,0,2,NaN
3,334455,1,2024-01-01 15:09:00,1,120.0,1,NaN,NaN,NaN,"(53.330332, -6.35966)","(53.330896, -6.331675)",1865.507988,NaN,False,0,0,3,NaN
4,334455,1,2024-01-01 15:12:00,1,180.0,1,NaN,NaN,NaN,"(53.329767, -6.387645)","(53.330332, -6.35966)",1865.536334,NaN,False,0,0,4,NaN
5,334455,1,2024-01-01 15:16:00,1,240.0,1,NaN,NaN,NaN,"(53.329203, -6.415631)","(53.329767, -6.387645)",1865.623771,NaN,False,0,0,5,NaN
6,334455,1,2024-01-01 15:17:00,1,60.0,1,NaN,NaN,NaN,"(53.328638, -6.443616)","(53.329203, -6.415631)",1865.585530,NaN,False,0,0,6,NaN
7,334455,1,2024-01-01 15:19:00,1,120.0,1,15.0,NaN,NaN,"(53.328074, -6.471601)","(53.328638, -6.443616)",1865.606380,NaN,False,1,1,7,8.333333
8,334455,1,2024-01-01 15:24:00,1,300.0,1,NaN,15.0,NaN,"(53.327509, -6.499586)","(53.328074, -6.471601)",1865.634726,NaN,False,0,1,8,8.333333
9,334455,1,2024-01-01 15:27:00,1,180.0,1,19.0,NaN,NaN,"(53.326945, -6.527571)","(53.327509, -6.499586)",1865.655575,NaN,False,1,2,9,10.185185


In [478]:
#max(sample['od_distance'])

In [479]:
sample.loc[sample.trip_section==0]['gps_distance']

KeyError: 'gps_distance'

In [127]:
first_val

id      trip_id
334455  1           15.0
        2          149.0
Name: curr_od_km, dtype: float64

In [483]:
sample.sort_values(['id', 'trip_id','timestamp', 'event_sec_id']).groupby(['id', 'trip_id'])['curr_od_km'].first(numeric_only=True)

KeyError: 'event_sec_id'

In [106]:
sample.sort_values(['id', 'trip_id','timestamp', 'event_sec_id']).groupby(['id', 'trip_id'])['curr_od_km']\
                .first(numeric_only=True) <= sample.sort_values(['id', 'trip_id','timestamp', 'event_sec_id'])\
                .groupby(['id', 'trip_id'])['curr_od_km'].min(numeric_only=True)

id      trip_id
334455  1          False
        2          False
Name: curr_od_km, dtype: bool

In [103]:
print(fact_tables[0].sort_values(['id', 'trip_id','timestamp', 'event_sec_id']).groupby(['id', 'trip_id'])['curr_od_km'].first(numeric_only=True))
print(fact_tables[0].sort_values(['id', 'trip_id','timestamp', 'event_sec_id']).groupby(['id', 'trip_id'])['curr_od_km'].min(numeric_only=True))


id      trip_id
334455  1           15.0
        2          149.0
Name: curr_od_km, dtype: float64
id      trip_id
334455  1          0.0
        2          2.0
Name: curr_od_km, dtype: float64


In [ ]:
fact_tables[0].sort_values(['id', 'trip_id','timestamp', 'event_sec_id']).groupby(['id', 'trip_id']).first(numeric_only=True)

In [57]:
distance.distance((53.332025,-6.275705),(53.331461,-6.30369)).m

1865.4587901995974

In [58]:
fact_tables[2]

,id,trip_id,timestamp,event_sec_id,timespent,ignition_state,curr_od_km,prev_od_km,od_distance,current_loc,prev_loc,gps_distance,fuel_%
0,334457,1,2024-01-01 10:00:00,1,NaN,1,1.25,NaN,NaN,"(53.679305, -7.297454)",NaN,NaN,100.0
1,334457,1,2024-01-01 10:02:13,1,133.0,1,2.50,1.25,1250.0,"(53.670328, -7.308877)","(53.679305, -7.297454)",1252.250939,NaN
2,334457,1,2024-01-01 10:03:53,1,100.0,1,3.75,2.50,1250.0,"(53.661352, -7.3203)","(53.670328, -7.308877)",1252.257685,NaN
3,334457,1,2024-01-01 10:06:11,1,138.0,1,5.00,3.75,1250.0,"(53.652375, -7.331723)","(53.661352, -7.3203)",1252.442019,NaN
4,334457,1,2024-01-01 10:06:22,1,11.0,1,6.25,5.00,1250.0,"(53.643399, -7.343146)","(53.652375, -7.331723)",1252.448783,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,334457,2,2024-01-02 16:59:06,1,39.0,1,318.24,316.71,1530.0,"(53.975834, -6.519412)","(53.977359, -6.542739)",1539.927351,NaN
476,334457,2,2024-01-02 17:00:03,1,57.0,1,319.77,318.24,1530.0,"(53.974309, -6.496084)","(53.975834, -6.519412)",1540.048112,NaN
477,334457,2,2024-01-02 17:00:39,1,36.0,1,321.30,319.77,1530.0,"(53.972783, -6.472756)","(53.974309, -6.496084)",1540.115948,NaN
478,334457,2,2024-01-02 17:01:20,1,41.0,1,322.83,321.30,1530.0,"(53.971258, -6.449429)","(53.972783, -6.472756)",1540.094021,NaN


In [59]:
fact_tables[0]['prev_loc']

0                         NaN
1      (53.332025, -6.275705)
2       (53.331461, -6.30369)
3      (53.330896, -6.331675)
4       (53.330332, -6.35966)
                ...          
199    (53.329767, -6.387645)
200     (53.330332, -6.35966)
201    (53.330896, -6.331675)
202     (53.331461, -6.30369)
203    (53.332025, -6.275705)
Name: prev_loc, Length: 204, dtype: object

In [60]:
#fact_tables[0]['distance'] = distance.distance((fact_tables[0]['current_lat'],fact_tables[0]['current_lon']), (fact_tables[0]['prev_lat'],fact_tables[0]['prev_lon']) )

In [61]:

newport_ri = (41.49008, -71.312796)
cleveland_oh = (41.499498, -81.695391)
print(distance.distance(newport_ri, cleveland_oh).km)
866.4554329098687
newport_ri = (41.49008, -71.312796, 100)
cleveland_oh = (41.499498, -81.695391, 100)
print(distance.distance(newport_ri, cleveland_oh).km)
866.4554329098687

866.4554329098687
866.4554329098687


866.4554329098687

In [ ]:
    ex4_pandas.sort_values("Date")
    .groupby("ticker")["closing_price"]

In [ ]:
print(fact_tables[3].head(50))

In [ ]:
fact_tables[0].to_csv('first_sample.csv')
fact_tables[1].to_csv('second_sample.csv')
fact_tables[2].to_csv('third_sample.csv')
fact_tables[3].to_csv('fourth_sample.csv')
fact_tables[4].to_csv('fifth_sample.csv')


In [ ]:
for fact in fact_tables: 
    order_and_ignition_fill(fact)

In [ ]:
fact_tables[0]

In [ ]:
first_sample.duplicated(subset = ['id', 'timestamp', 'seq_number'])

In [ ]:
first_sample.to_csv('first_sample.csv')

In [ ]:
first_sample

In [ ]:
first_sample.columns = facts_cals
first_sample


In [ ]:
first_sample.columns

In [ ]:
fifth_sample

In [ ]:
fourth_sample.columns

In [ ]:
dataframes = []

s=re.fullmatch('\d*\.xlsx', '334455.xlsx')
if s is not None: 
    print(s.string)
    pd.pandas.read_excel(, sheet_name=0, *, 
                         header=0, names=None, index_col=None, usecols=None, 
                         dtype=None, engine=None, converters=None, true_values=None, 
                         false_values=None, skiprows=None, nrows=None, na_values=None, 
                         keep_default_na=True, na_filter=True, verbose=False, 
                         parse_dates=False, date_parser=_NoDefault.no_default, 
                         date_format=None, thousands=None, decimal='.', comment=None, 
                         skipfooter=0, storage_options=None, dtype_backend=_NoDefault.no_default, engine_kwargs=None)